# Generate Infrastructure report

## This generates the infrastructure report for a given country. Follow the instructions below

In [ ]:
# Imports

In [1]:
from IPython.display import display, HTML, clear_output, FileLink
import ipywidgets as widgets
import os,sys
import pandas as pd
import warnings
import zipfile
import geopandas as gpd 
import fiona 
import numpy as np
#import folium
import math
import pickle
import matplotlib.pyplot as plt
from shapely.geometry import Point, LineString, Polygon
import networkx as nx
#from folium.plugins import HeatMap
from pylatex import Document, Section, Subsection, Tabular, Command
from pylatex import Math, TikZ, Axis, Plot, Figure, Matrix, Alignat, SubFigure
from pylatex import PageStyle, Head, Foot, MiniPage, \
    StandAloneGraphic, MultiColumn, Tabu, LongTabu, LargeText, MediumText, \
    LineBreak, NewPage, Tabularx, TextColor, simple_page_number, Itemize, Enumerate, Hyperref, Package, \
    TikZ, TikZNode, TikZCoordinate, TikZOptions, NewLine, VerticalSpace
from pylatex.utils import italic, bold, NoEscape, escape_latex
import base64
import matplotlib.patches as mpatches
from matplotlib.collections import PatchCollection
from matplotlib.patches import Patch
from haversine import haversine, Unit
import contextlib
import io
import logging
from typing import Any, List, Generator, IO
from azure.storage.blob import BlobServiceClient
import shutil

In [2]:
#import sys
#!{sys.executable} -m pip install haversine

In [3]:
# Some settings

In [4]:
pd.set_option('display.max_columns', None)

warnings.filterwarnings('ignore')

In [5]:
# Constants

In [6]:
COLORS_CONNECT = {'Yes':'green','No':'red','YES':'green','NO':'red'}
COLOR_UNKOWN = 'blue'
patches_connect = [mpatches.Patch(color='green', label='Yes'),mpatches.Patch(color='red', label='No'),mpatches.Patch(color='blue', label='Unknown')]
COLORS_DISTS = ['blue','green','yellow','orange','red']
fiber_dist_plot_threshold = 10 #kms
cell_dist_plot_threshold = 5 #kms
p2p_dist_plot_threshold = 10 #kms
los_buffer = 5 #Actual uses this - to add to intermediate points' real height
COLORS_TECHS = {'Fiber':'blue','P2P':'orange','cellular':'red','satellite':'lightblue'}
#blob
ADLS_CONNECTION_STRING = 'DefaultEndpointsProtocol=https;AccountName=saunigiga;AccountKey=HcvYWMRHu6y+C3FKMLitMl50lA6Bd9AKiCV8GI2TdhUSbKFD4SSBf71ZE7w6+wmDRcGbcPc2Vbv4oDar/eJGJA==;EndpointSuffix=core.windows.net'#os.environ.get("ADLS_CONNECTION_STRING", "")
ADLS_CONTAINER = "giga"

# We use this logger to disable logs from the blob storage module
logger = logging.getLogger("adls_custom_logger")
logger.disabled = True

DEFAULT_TOWER_HEIGHT = 25

In [7]:
# Functions

In [8]:
def readCodes(fn):
    d = {}
    dfc = pd.read_csv(fn)
    for index,row in dfc.iterrows():
        d[row['Code']] = row['Country']
        
    return d

def plot_country_map(gdf):
    # Load GeoJSON data
    #gdf = gpd.read_file(fn)

    # Create the plot
    fig, ax = plt.subplots(1, 1)
    gdf.plot(ax=ax,edgecolor='black', facecolor='lightgrey')
    ax.axis('off')

    # Save the figure
    plt.savefig('aux_files/country_map.png', bbox_inches='tight', pad_inches=0)
    plt.close(fig)

def plot_piechart_nans(df,column_name):
    # Define the data
    labels = ['Unknown', 'Information Available']
    num_nan = df[column_name].isna().sum()
    sizes = [num_nan,len(df)-num_nan]

    fig, ax = plt.subplots()
    # Create a pie chart
    ax.pie(sizes, labels=labels, autopct='%1.1f%%',shadow=True, startangle=90)

    # Equal aspect ratio ensures that the pie is drawn as a circle
    ax.axis('equal')  

    plt.savefig('aux_files/'+column_name+'_nan_pie.png')
    
def plot_piechart_counts_with_dict(df,column_name,d):
    # Define the data
    counts = df[column_name].value_counts()

    # Convert the index to a list to get the unique values
    labels_pre = counts.index.tolist()

    # Convert the values to a list to get the counts
    sizes_pre = counts.values.tolist()
    
    # Merge from dict
    index = {}
    index_pre = {}
    labels = []
    sizes = []
    j = 0
    for i in range(len(labels_pre)):
        label = labels_pre[i]
        if label not in d:
            labels.append(label)
            sizes.append(sizes_pre[i])
            index[label] = j
            j += 1
        else:
            index_pre[label] = i
            
    for key in d:
        new_label = d[key]
        sizes[index[new_label]] += sizes_pre[index_pre[key]]

    fig, ax = plt.subplots()
    # Create a pie chart
    ax.pie(sizes, labels=labels, autopct='%1.1f%%',shadow=True, startangle=90)

    # Equal aspect ratio ensures that the pie is drawn as a circle
    plt.axis('equal')  

    plt.savefig('aux_files/'+column_name+'_pie.png')
    
def map_connectivity_schools(gdf,df):
    # Load GeoJSON data
    #gdf = gpd.read_file(fn)

    # Create the plot
    fig, ax = plt.subplots(1, 1)
    gdf.plot(ax=ax,edgecolor='black', facecolor='lightgrey')
    ax.axis('off')
    #plt.title('School connectivity status')
    fig.text(0.5, 0.01, "School connectivity status", ha="center", fontsize=10)
    colors_lons = {}
    colors_lats = {}
    for key in COLORS_CONNECT:
        colors_lons[COLORS_CONNECT[key]] = []
        colors_lats[COLORS_CONNECT[key]] = []
    colors_lons[COLOR_UNKOWN] = []
    colors_lats[COLOR_UNKOWN] = []
        
    
    for index,row in df.iterrows():
        color = COLOR_UNKOWN
        if not pd.isnull(row['connectivity']):
            color = COLORS_CONNECT[row['connectivity']]
        colors_lons[color].append(row['lon'])
        colors_lats[color].append(row['lat'])
        
    for color in colors_lats:
        ax.scatter(colors_lons[color], colors_lats[color], color=color, s=1) 

    # Set the legend
    leg = ax.legend(handles=patches_connect, bbox_to_anchor=(1.05, 1),
                         loc='upper left', borderaxespad=0., title='Status')

    # Save the figure
    plt.savefig('aux_files/connectivity_map.png', bbox_inches='tight', pad_inches=0)
    plt.close(fig)
    
    
def plot_map_distances(gdf,df,dists,brackets,tech):
    # Load GeoJSON data
    #gdf = gpd.read_file(fn)

    # Create the plot
    fig, ax = plt.subplots(1, 1)
    gdf.plot(ax=ax,edgecolor='black', facecolor='lightgrey')
    ax.axis('off')
    #plt.title('School connectivity status')
    fig.text(0.5, 0.01, "Distance to fiber nodes", ha="center", fontsize=10)
    
    colors_lons = {}
    colors_lats = {}
    for key in COLORS_DISTS:
        colors_lons[key] = []
        colors_lats[key] = []
    colors_lons['black'] = []
    colors_lats['black'] = []
    
    for idx, row in df.iterrows():
        tooltip = row['giga_id_school']
        #brackets has length 5ç
        if len(dists[tooltip])>0:
            d = min(dists[tooltip].values())
        else:
            d = math.inf
        color = "black"
        for i in range(len(brackets)):
            if d<brackets[i]:
                color = COLORS_DISTS[i]
                break
        colors_lons[color].append(row['lon'])
        colors_lats[color].append(row['lat'])
        
    for color in colors_lats:
        ax.scatter(colors_lons[color], colors_lats[color], color=color, s=1) 
        # Set the legend
        
    patches = [mpatches.Patch(color=COLORS_DISTS[0], label='0 - '+str(brackets[0]))]
    for i in range(1,len(brackets)):
        patches.append(mpatches.Patch(color=COLORS_DISTS[i], label=str(brackets[i-1])+' - '+ str(brackets[i])))
    leg = ax.legend(handles=patches, bbox_to_anchor=(1.05, 1),
                         loc='upper left', borderaxespad=0., title='Distance, kms')

    # Save the figure
    plt.savefig('aux_files/'+tech+'_dists_map.png', bbox_inches='tight', pad_inches=0)
    plt.close(fig)
    
def plot_dist_distributions(data,threshold,pctg_th,entity):

    fig, ax1 = plt.subplots()

    # Set x-axis limit to start from 0
    ax1.set_xlim(0, max(data))

    # Create histogram on the first Y axis
    n, bins, patches = ax1.hist(data, bins=100, density=True, alpha=0.6, color='g', label='Histogram of school distances')
    ax1.set_ylabel('Percentage of schools')
    ax1.yaxis.set_major_formatter(plt.FuncFormatter(lambda x, _: '{:.0f}'.format(x*100)))  # Multiply by 100

    # Create a second Y axis
    ax2 = ax1.twinx()
    ax2.set_ylim(0, 110)


    # Create cumulative distribution on the second Y axis
    hist_data, _ = np.histogram(data, bins, density=True)
    y = np.cumsum(hist_data)/np.sum(hist_data)
    y = np.insert(y,0,0)
    
    ax2.plot(bins, y*100, 'r-', linewidth=1.5, label='Cumulative distribution of school distances') # Cumulative distribution
    ax2.fill_between(bins, y*100, alpha=0.2, color='r') # Shading under curve

    ax2.yaxis.set_major_formatter(plt.FuncFormatter('{:.0f}%'.format))

    # Add a vertical line at threshold
    ax1.axvline(threshold, color='b', linestyle='dashed', linewidth=2, label=str(threshold)+' distance threshold')
    # Add a horizontal line
    ax2.axhline(pctg_th, color='purple', linestyle='dashdot', linewidth=2, label='% of schools within '+str(threshold)+' of the closest '+ entity)

    # Add legend
    fig.legend(loc="upper center", bbox_to_anchor=(0.5,1), bbox_transform=ax1.transAxes, fontsize='x-small')
    ax1.set_xlabel('Distance to closest '+ entity+', kms')
    plt.title('Distribution of distances from schools to closest '+ entity, fontsize='medium')

    #plt.savefig('aux_files/fiber_dists_map.png', bbox_inches='tight', pad_inches=0)
    plt.savefig('aux_files/dist_distributions_'+entity.split()[0]+'.png')
    plt.close(fig)
    
def less_than_dists_with_condition(dists,th,df,idcolumn,column,value):
    counts = 0
    for key in dists:
        if len(dists[key])>0:
            sorted_dict = sorted(dists[key].items(), key=lambda item: item[1])
            for item in sorted_dict:
                (tower_id,d) = item
                if d<th:
                    row = df[df[idcolumn]==tower_id].iloc[0]
                    if value in row[column]:
                        counts += 1
                        break
                else:
                    break
            
    return counts

def plot_signal_strength(gdf,df_cell,substrings):
    #gdf = gpd.read_file(fn_country)
    country_boundary = gdf['geometry'].unary_union
    
    # Create the plot
    fig, ax = plt.subplots()
    gdf.plot(ax=ax,edgecolor='black', facecolor='white')
    ax.axis('off')

    patches = [Patch(facecolor='#0000FF', edgecolor='#0000FF', label='Strong'),
                   Patch(facecolor='#6666FF', edgecolor='#6666FF', label='Average'),
                   Patch(facecolor='#CCCCFF', edgecolor='#CCCCFF', label='Weak')]


    df = df_cell.copy()
    pattern = '|'.join(substrings)
    df = df[df['technologies'].str.contains(pattern)]
    
    df['geometry'] = df.apply(lambda row: Point(row.lon, row.lat), axis=1)
    gdf_cell = gpd.GeoDataFrame(df, geometry='geometry')
    within_country = gdf_cell[gdf_cell.geometry.within(country_boundary)]

    # Generate circles for each point in the DataFrame
    circles_outer = []
    circles_outer2 = []
    circles_inner = []
    # Convert coordinates from geopandas to matplotlib
    coord_x = within_country.geometry.x
    coord_y = within_country.geometry.y

    for x, y in zip(coord_x, coord_y):
        # Outer, lighter circle
        circle_outer2 = mpatches.Circle((x,y), 0.23, edgecolor='none', facecolor='#CCCCFF', alpha=0.2)
        circles_outer2.append(circle_outer2)
    
        # Outer, lighter circle
        circle_outer = mpatches.Circle((x,y), 0.15, edgecolor='none', facecolor='#6666FF', alpha=0.5)
        circles_outer.append(circle_outer)

        # Inner, darker circle
        circle_inner = mpatches.Circle((x,y), 0.08, edgecolor='none', facecolor='#0000FF', alpha=1.0)
        circles_inner.append(circle_inner)


    # Create a collection of these circles and add it to the plot
    p_outer = PatchCollection(circles_outer, match_original=True)
    ax.add_collection(p_outer)

    p_outer2 = PatchCollection(circles_outer2, match_original=True)
    ax.add_collection(p_outer2)

    p_inner = PatchCollection(circles_inner, match_original=True)
    ax.add_collection(p_inner)
    
    # Set aspect
    ax.set_aspect('equal')

    # Add legend to the plot
    leg = ax.legend(handles=patches, bbox_to_anchor=(1.05, 1),
                         loc='upper left', borderaxespad=0., title='4G coverage area strength')

    plt.savefig('aux_files/cell_strength.png', bbox_inches='tight', pad_inches=0)
    plt.close(fig)
    
def plot_visibility_map(gdf,visible_towers,df_schools,df_cell,brackets):
    
    #gdf = gpd.read_file(fn_country)
    
    # Create the plot
    fig, ax = plt.subplots()
    gdf.plot(ax=ax,edgecolor='black', facecolor='lightgrey')
    ax.axis('off')
    fig.text(0.5, 0.01, "Cell tower visibility", ha="center", fontsize=10)
    
    lines = []
    colors = []
    for key in visible_towers:
        if len(visible_towers[key])>0:
            sorted_pairs = sorted(visible_towers[key], key=lambda pair: pair[1])
            tower_id, d = sorted_pairs[0]
            if d<brackets[0]:
                color = COLORS_DISTS[0]
            elif d<brackets[1]:
                color = COLORS_DISTS[1]
            elif d<brackets[2]:
                color = COLORS_DISTS[2]
            elif d<brackets[3]:
                color = COLORS_DISTS[3]
            elif d<brackets[4]:
                color = COLORS_DISTS[4]
            else:
                color = 'black'
            colors.append(color)
            master_row = df_schools[df_schools['giga_id_school']==key].iloc[0]
            cell_row = df_cell[df_cell['tower_id']==tower_id].iloc[0]
            point1 = Point(master_row['lon'], master_row['lat'])
            point2 = Point(cell_row['lon'], cell_row['lat'])
            line = LineString([point1, point2])
            lines.append(line)
            
    # Create a GeoDataFrame from the lines
    lines_gdf = gpd.GeoDataFrame(geometry=lines)

    # Add the colors to the GeoDataFrame
    lines_gdf['color'] = colors
    
    # Plot the lines with color based on 'color' column
    for x in range(len(lines_gdf)):
        #gpd.plotting.plot_linestring(ax, lines_gdf.geometry.iloc[x], color=lines_gdf.color.iloc[x])
        line = lines_gdf.geometry.iloc[x]
        color = lines_gdf.color.iloc[x]
        x, y = line.xy
        ax.plot(x, y, color=color, lw=1)
        
    patches = [mpatches.Patch(color=COLORS_DISTS[0], label='0 - '+str(brackets[0]))]
    for i in range(1,len(brackets)):
        patches.append(mpatches.Patch(color=COLORS_DISTS[i], label=str(brackets[i-1])+' - '+ str(brackets[i])))
    leg = ax.legend(handles=patches, bbox_to_anchor=(1.05, 1),
                         loc='upper left', borderaxespad=0., title='Closest visible tower, kms')
    
    ax.set_aspect('equal')
    plt.savefig('aux_files/visibility.png', bbox_inches='tight', pad_inches=0)
    plt.close(fig)
    
def is_visibleActual(elevation_profile):
    
    coords_array = [(p[0],p[1]) for p in elevation_profile]
    elevation_array = [p[2] for p in elevation_profile]
    distance_array = [haversine(coords_array[0], coords_array[i], unit=Unit.METERS) for i in range(len(coords_array))]

    coordinates_combined = list(zip(distance_array, elevation_array))
    start_and_endpoints = LineString(
            [Point(coordinates_combined[0]), Point(coordinates_combined[-1])]
        )
    elevation_profile = LineString(coordinates_combined[1:-1])

    if start_and_endpoints.intersects(elevation_profile):
        return False
    else:
        return True
    
def plot_visibility_distributions(data):

    fig, ax1 = plt.subplots()

    # Set x-axis limit to start from 0
    ax1.set_xlim(0, max(data))

    # Create histogram on the first Y axis
    n, bins, patches = ax1.hist(data, bins=100, density=True, alpha=0.6, color='g', label='Histogram of school distances')
    ax1.set_ylabel('Percentage of schools')
    ax1.yaxis.set_major_formatter(plt.FuncFormatter(lambda x, _: '{:.0f}'.format(x*100)))  # Multiply by 100

    # Create a second Y axis
    ax2 = ax1.twinx()
    ax2.set_ylim(0, 110)


    # Create cumulative distribution on the second Y axis
    hist_data, _ = np.histogram(data, bins, density=True)
    y = np.cumsum(hist_data)/np.sum(hist_data)
    y = np.insert(y,0,0)
    
    ax2.plot(bins, y*100, 'r-', linewidth=1.5, label='Cumulative distribution of school distances') # Cumulative distribution
    ax2.fill_between(bins, y*100, alpha=0.2, color='r') # Shading under curve

    ax2.yaxis.set_major_formatter(plt.FuncFormatter('{:.0f}%'.format))

    # Add legend
    fig.legend(loc="upper center", bbox_to_anchor=(0.5,1), bbox_transform=ax1.transAxes, fontsize='x-small')
    ax1.set_xlabel('Distance to closest visible tower, kms')
    plt.title('Distribution of distances from schools to closest visible tower', fontsize='medium')

    plt.savefig('aux_files/dist_distributions_visible.png')
    plt.close(fig)
    
def count_lists_of_at_least_size(dictionary, size):
    count = 0
    for key in dictionary:
        if len(dictionary[key]) >= size:
            count += 1
    return count

def get_avg_visible(visible_towers):
    sumv = 0
    counts = 0
    for key in visible_towers:
        if len(visible_towers[key])>0:
            sorted_pairs = sorted(visible_towers[key], key=lambda pair: pair[1])
            tower_id, d = sorted_pairs[0]
            sumv += d
            counts += 1
        
    return sumv/float(counts)

def get_fiber_graph(df_fiber,school_dists,fnode_dists,th,valid_schools,extra_fiber_nodes):
    G = nx.Graph()
    G.add_node('meta',label='meta')
    # Connect with all fiber nodes and if any all fiber connected schools
    for index,row in df_fiber.iterrows():
        fid = row['coordinate_id']
        G.add_node(fid,label='fnode')
        G.add_edge('meta',fid,w=0.0)
        
    for sc in extra_fiber_nodes:
        fid = sc
        G.add_node(fid,label='fnode-school')
        G.add_edge('meta',fid,w=0.0)
        
    for key in valid_schools:
        if not G.has_node(key):
            G.add_node(key,label='school')
        for key2 in school_dists[key]:
            d = school_dists[key][key2]
            if d<=th:
                if not G.has_node(key2):
                    G.add_node(key2,label='school')
                G.add_edge(key,key2,w=d)
                    
        for key2 in fnode_dists[key]:
            d = fnode_dists[key][key2]
            if d<=th:
                G.add_edge(key2,key,w=d)

    # remove isolates
    G.remove_nodes_from(list(nx.isolates(G)))
    
    # We need to find the connected component that contains 'meta'
    ccs_G = [c for c in sorted(nx.connected_components(G), key=len, reverse=True)]
    G_trimmed = None
    for i in range(len(ccs_G)):
        if 'meta' in ccs_G[i]:
            G_trimmed = G.subgraph(ccs_G[i]).copy() 
            break
            
    return G_trimmed

def get_MST(G):
    return nx.minimum_spanning_tree(G,weight='w')

def get_total_fiber(T,correction_coef):
    total = 0.0
    for node1,node2,data in T.edges(data=True):
        if node1!='meta' and node2!='meta':
            total += data['w']
            
    return total*correction_coef

def get_total_fiber_and_num_schools(T,correction_coef):
    total = 0.0
    schools = set()
    for node1,node2,data in T.edges(data=True):
        if node1!='meta' and node2!='meta':
            total += data['w']
            
        if T.nodes[node1]['label']=='school':
            schools.add(node1)
        if T.nodes[node2]['label']=='school':
            schools.add(node2)
            
    return total*correction_coef,len(schools)

def plot_map_tree(gdf,df,df2,T,th,kms,cc):
    
    coords = {}
    for node,data in T.nodes(data=True):
        if node!='meta':
            if data['label']=='school' or data['label']=='fnode-school':
                f_df = df[df['giga_id_school'] == node]
            else:
                f_df = df2[df2['coordinate_id'] == node]
            coords[node] = Point(f_df['lon'].values[0],f_df['lat'].values[0])
    
    #gdf = gpd.read_file(fn_country)
    
    # Create the plot
    fig, ax = plt.subplots()
    gdf.plot(ax=ax,edgecolor='black', facecolor='lightgrey')
    ax.axis('off')
    fig.text(0.5, 0.01, f"{kms:.1f} kms of fiber (max = {th*cc:.1f})", ha="center", fontsize=8)
    
    lines = []
    for node1,node2,data in T.edges(data=True):
        if node1!='meta' and node2!='meta':
            point1 = coords[node1]
            point2 = coords[node2]
            line = LineString([point1, point2])
            lines.append(line)

    # Create a GeoDataFrame from the lines
    lines_gdf = gpd.GeoDataFrame(geometry=lines)
    
    # Plot the lines with color based on 'color' column
    for x in range(len(lines_gdf)):
        #gpd.plotting.plot_linestring(ax, lines_gdf.geometry.iloc[x], color=lines_gdf.color.iloc[x])
        line = lines_gdf.geometry.iloc[x]
        x, y = line.xy
        ax.plot(x, y, color='blue', lw=1)
        
    ax.set_aspect('equal')
    plt.savefig(f'aux_files/fiber_paths_{th}.png', bbox_inches='tight', pad_inches=0)
    plt.close(fig) 
    
def fill_techs(df_schools,df_cell,cell_dists,tech,visible_towers,T,th,valid_schools):
    cell_used_i = []
    p2p_used_i = []
    vsat_used_i = []
    for index,row in df_schools.iterrows():
        if not T.has_node(row['giga_id_school']) and row['giga_id_school'] in valid_schools:
            in_cell = False
            sorted_dict = sorted(cell_dists[row['giga_id_school']].items(), key=lambda item: item[1])
            for item in sorted_dict:
                (tower_id,d) = item
                if d<th:
                    cell_row = df_cell[df_cell['tower_id']==tower_id].iloc[0]
                    if len(tech)==3:
                        cell_used_i.append(row['giga_id_school'])
                        in_cell = True
                        break
                    elif len(tech)==1:
                        if tech[0] in cell_row['technologies']:
                            cell_used_i.append(row['giga_id_school'])
                            in_cell = True
                            break
                    else:
                        if tech[0] in cell_row['technologies'] or tech[1] in cell_row['technologies']:
                            cell_used_i.append(row['giga_id_school'])
                            in_cell = True
                            break
                else:
                    break
            if not in_cell:
                if len(visible_towers)>0 and row['giga_id_school'] in visible_towers and len(visible_towers[row['giga_id_school']])>0:
                    p2p_used_i.append(row['giga_id_school'])
                else:
                    vsat_used_i.append(row['giga_id_school'])
                    
    return cell_used_i, p2p_used_i, vsat_used_i

def plot_map_4(gdf,df,T,cell,p2p,vsat,th):
    
    # Load GeoJSON data
    #gdf = gpd.read_file(fn_country)

    # Create the plot
    fig, ax = plt.subplots(1, 1)
    gdf.plot(ax=ax,edgecolor='black', facecolor='lightgrey')
    ax.axis('off')
    #plt.title('School connectivity status')
    fig.text(0.5, 0.01, "School technologies", ha="center", fontsize=10)

    # Generate base map
    #print('fiber')
    lons = []
    lats = []
    for node,data in T.nodes(data=True):
        if data['label']=='school' or data['label']=='fnode-school':
            f_df = df[df['giga_id_school'] == node]
            lons.append(f_df['lon'].values[0])
            lats.append(f_df['lat'].values[0])
    ax.scatter(lons, lats, color=COLORS_TECHS['Fiber'], s=1) 
    #print('cell')
    lons = []
    lats = []
    for node in cell:
        f_df = df[df['giga_id_school'] == node]
        lons.append(f_df['lon'].values[0])
        lats.append(f_df['lat'].values[0])
    ax.scatter(lons, lats, color=COLORS_TECHS['cellular'], s=1)    
    #print('p2p')
    lons = []
    lats = []
    for node in p2p:
        f_df = df[df['giga_id_school'] == node]
        lons.append(f_df['lon'].values[0])
        lats.append(f_df['lat'].values[0])
    ax.scatter(lons, lats, color=COLORS_TECHS['P2P'], s=1)        
    #print('vsat')
    lons = []
    lats = []
    for node in vsat:
        f_df = df[df['giga_id_school'] == node]
        lons.append(f_df['lon'].values[0])
        lats.append(f_df['lat'].values[0])
    ax.scatter(lons, lats, color=COLORS_TECHS['satellite'], s=1)    
    patches = []
    #print('patches')
    for key in COLORS_TECHS:
        patches.append(mpatches.Patch(color=COLORS_TECHS[key], label=key))
        
    leg = ax.legend(handles=patches, bbox_to_anchor=(1.05, 1),
                         loc='upper left', borderaxespad=0., title='Schools to be connected with:')

    # Save the figure
    plt.savefig('aux_files/scenarios_'+str(th)+'.png', bbox_inches='tight', pad_inches=0)
    plt.close(fig)
    
def fill_techs_no_fiber(df_schools,df_cell,cell_dists,tech,visible_towers,th,valid_schools):
    cell_used_i = []
    p2p_used_i = []
    vsat_used_i = []
    for index,row in df_schools.iterrows():
        if row['giga_id_school'] in valid_schools:
            in_cell = False
            sorted_dict = sorted(cell_dists[row['giga_id_school']].items(), key=lambda item: item[1])
            for item in sorted_dict:
                (tower_id,d) = item
                if d<th:
                    cell_row = df_cell[df_cell['tower_id']==tower_id].iloc[0]
                    if len(tech)==3:
                        cell_used_i.append(row['giga_id_school'])
                        in_cell = True
                        break
                    elif len(tech)==1:
                        if tech[0] in cell_row['technologies']:
                            cell_used_i.append(row['giga_id_school'])
                            in_cell = True
                            break
                    else:
                        if tech[0] in cell_row['technologies'] or tech[1] in cell_row['technologies']:
                            cell_used_i.append(row['giga_id_school'])
                            in_cell = True
                            break
                else:
                    break
            if not in_cell:
                if len(visible_towers)>0 and row['giga_id_school'] in visible_towers and len(visible_towers[row['giga_id_school']])>0:
                    p2p_used_i.append(row['giga_id_school'])
                else:
                    vsat_used_i.append(row['giga_id_school'])
                    
    return cell_used_i, p2p_used_i, vsat_used_i

def plot_connectivity_scenarios(xs_techs,ys_techs,maxy):

    fig, ax = plt.subplots()
    ax.set_facecolor('lightgray')
    ax.grid(color='white')
    ax.set_xticks([0, 5, 10, 15, 20])
    ax.set_xlim(0, 25)
    ax.set_ylim(0, maxy) 
    for tech in xs_techs:
        ax.plot(xs_techs[tech],ys_techs[tech],color=COLORS_TECHS[tech],label=tech)

    # Add legend
    fig.legend(loc="upper center", bbox_to_anchor=(0.5,0.5), bbox_transform=ax.transAxes, fontsize='x-small')
    ax.set_xlabel('Maximum distance between schools connected by fiber, kms')
    plt.title('Distribution of schools by technologies', fontsize='medium')
    ax.set_ylabel('Number of schools')

    plt.savefig('aux_files/schools_techs.png')
    plt.close(fig)
    
def create_accordion(titles,tab):    
    if tab=='Electricity':
        ccsc = widgets.FloatText(value=0.0, description='Solar cost (USD/Watt)', style= {'description_width': 'initial'})
        ocpk = widgets.FloatText(value=0.0, description='Cost per kWh', style= {'description_width': 'initial'})
        xprs = widgets.FloatText(value=0.0, description='Power required per school (Watts)', style= {'description_width': 'initial'})
        vbox1 = widgets.VBox([widgets.HTML(f'<b>Capex</b>'),ccsc])
        vbox2 = widgets.VBox([widgets.HTML(f'<b>Opex</b>'),ocpk])
        vbox3 = widgets.VBox([xprs])
        hbox = widgets.HBox([vbox1,vbox2])
        accordion = widgets.Accordion(children=[hbox,vbox3])
        accordion.set_title(0, titles[0])
        accordion.set_title(1, titles[1])
        return accordion
        
        
    
    ocpm = widgets.FloatText(value=0.0, description='Cost per Mbps/year', style= {'description_width': 'initial'})  
    omy = widgets.FloatText(value=0.0, description='Maintenance yearly', style= {'description_width': 'initial'})
    otb = widgets.FloatText(value=0.0, description='Transit Mbps/year', style= {'description_width': 'initial'})
    xapr = widgets.FloatText(value=0.0, description='Annual power required (KWh)', style= {'description_width': 'initial'})
    if tab=='Fiber':
        csc = widgets.FloatText(value=0.0, description='Setup cost', style= {'description_width': 'initial'})
        cpk = widgets.FloatText(value=0.0, description='Cost per km', style= {'description_width': 'initial'})
        opk = widgets.FloatText(value=0.0, description='Maintenance per km', style= {'description_width': 'initial'})
        vbox1 = widgets.VBox([widgets.HTML(f'<b>Capex</b>'),csc,cpk])
        vbox2 = widgets.VBox([widgets.HTML(f'<b>Opex</b>'),ocpm,omy,otb,opk])
        xml = widgets.FloatText(value=0.0, description='Max length', style= {'description_width': 'initial'})
        vbox3 = widgets.VBox([xapr,xml])
    elif tab=='Cellular':
        csc = widgets.FloatText(value=0.0, description='Setup cost', style= {'description_width': 'initial'})
        vbox1 = widgets.VBox([widgets.HTML(f'<b>Capex</b>'),csc])
        vbox2 = widgets.VBox([widgets.HTML(f'<b>Opex</b>'),ocpm,omy,otb])
        xml = widgets.FloatText(value=0.0, description='Max length', style= {'description_width': 'initial'})
        vbox3 = widgets.VBox([xapr,xml])
    elif tab=='P2P':
        cscs = widgets.FloatText(value=0.0, description='Setup cost - school', style= {'description_width': 'initial'})
        csct = widgets.FloatText(value=0.0, description='Setup cost - tower', style= {'description_width': 'initial'})
        vbox1 = widgets.VBox([widgets.HTML(f'<b>Capex</b>'),cscs,csct])
        vbox2 = widgets.VBox([widgets.HTML(f'<b>Opex</b>'),ocpm,omy,otb])
        xml = widgets.FloatText(value=0.0, description='Max length', style= {'description_width': 'initial'})
        vbox3 = widgets.VBox([xapr,xml])        
    else: #satellite
        csc = widgets.FloatText(value=0.0, description='Setup cost', style= {'description_width': 'initial'})
        vbox1 = widgets.VBox([widgets.HTML(f'<b>Capex</b>'),csc])
        vbox2 = widgets.VBox([widgets.HTML(f'<b>Opex</b>'),ocpm,omy,otb])
        vbox3 = widgets.VBox([xapr])
        
    hbox = widgets.HBox([vbox1,vbox2])
    accordion = widgets.Accordion(children=[hbox,vbox3])
    accordion.set_title(0, titles[0])
    accordion.set_title(1, titles[1])
    return accordion

def update_tabs_blob(path,iso3code):
    global tabs
    
    # create a dictionary to store the dataframes
    dfs_costs = {}

    # iterate over all files in the specified directory
    for filename in os.listdir(path):
        if filename.endswith(".csv"):  # check if the file is a CSV
            df = pd.read_csv(os.path.join(path, filename))  # read the CSV file into a dataframe
            dfs_costs[filename[:-4]] = df  # store the dataframe in the dictionary
            
    #fiber
    df_fc = dfs_costs['fiber_capex']
    tabs.children[0].children[0].children[0].children[1].value = df_fc[df_fc['Country']==iso3code]['Setup cost'].values[0]
    tabs.children[0].children[0].children[0].children[2].value = df_fc[df_fc['Country']==iso3code]['Cost per km'].values[0]
    df_fo = dfs_costs['fiber_opex']
    tabs.children[0].children[0].children[1].children[1].value = df_fo[df_fo['Country']==iso3code]['Cost per Mbps/year'].values[0]
    tabs.children[0].children[0].children[1].children[2].value = df_fo[df_fo['Country']==iso3code]['Maintenance yearly'].values[0]
    tabs.children[0].children[0].children[1].children[3].value = df_fo[df_fo['Country']==iso3code]['Transit Mbps/year'].values[0]
    tabs.children[0].children[0].children[1].children[4].value = df_fo[df_fo['Country']==iso3code]['Maintenance per km'].values[0]
    df_fx = dfs_costs['fiber_cstrs']
    tabs.children[0].children[1].children[0].value = df_fx[df_fx['Country']==iso3code]['Annual power required (KWh)'].values[0]
    tabs.children[0].children[1].children[1].value = df_fx[df_fx['Country']==iso3code]['Max length'].values[0]
    
    #cellular
    df_fc = dfs_costs['cell_capex']
    tabs.children[1].children[0].children[0].children[1].value = df_fc[df_fc['Country']==iso3code]['Setup cost'].values[0]
    df_fo = dfs_costs['cell_opex']
    tabs.children[1].children[0].children[1].children[1].value = df_fo[df_fo['Country']==iso3code]['Cost per Mbps/year'].values[0]
    tabs.children[1].children[0].children[1].children[2].value = df_fo[df_fo['Country']==iso3code]['Maintenance yearly'].values[0]
    tabs.children[1].children[0].children[1].children[3].value = df_fo[df_fo['Country']==iso3code]['Transit Mbps/year'].values[0]
    df_fx = dfs_costs['cell_cstrs']
    tabs.children[1].children[1].children[0].value = df_fx[df_fx['Country']==iso3code]['Annual power required (KWh)'].values[0]
    tabs.children[1].children[1].children[1].value = df_fx[df_fx['Country']==iso3code]['Max length'].values[0]
    
    #p2p
    df_fc = dfs_costs['p2p_capex']
    tabs.children[2].children[0].children[0].children[1].value = df_fc[df_fc['Country']==iso3code]['Setup cost - school'].values[0]
    tabs.children[2].children[0].children[0].children[2].value = df_fc[df_fc['Country']==iso3code]['Setup cost - tower'].values[0]
    df_fo = dfs_costs['p2p_opex']
    tabs.children[2].children[0].children[1].children[1].value = df_fo[df_fo['Country']==iso3code]['Cost per Mbps/year'].values[0]
    tabs.children[2].children[0].children[1].children[2].value = df_fo[df_fo['Country']==iso3code]['Maintenance yearly'].values[0]
    tabs.children[2].children[0].children[1].children[3].value = df_fo[df_fo['Country']==iso3code]['Transit Mbps/year'].values[0]
    df_fx = dfs_costs['p2p_cstrs']
    tabs.children[2].children[1].children[0].value = df_fx[df_fx['Country']==iso3code]['Annual power required (KWh)'].values[0]
    tabs.children[2].children[1].children[1].value = df_fx[df_fx['Country']==iso3code]['Max length'].values[0]
    
    #sat
    df_fc = dfs_costs['sat_capex']
    tabs.children[3].children[0].children[0].children[1].value = df_fc[df_fc['Country']==iso3code]['Setup cost'].values[0]
    df_fo = dfs_costs['sat_opex']
    tabs.children[3].children[0].children[1].children[1].value = df_fo[df_fo['Country']==iso3code]['Cost per Mbps/year'].values[0]
    tabs.children[3].children[0].children[1].children[2].value = df_fo[df_fo['Country']==iso3code]['Maintenance yearly'].values[0]
    tabs.children[3].children[0].children[1].children[3].value = df_fo[df_fo['Country']==iso3code]['Transit Mbps/year'].values[0]
    df_fx = dfs_costs['sat_cstrs']
    tabs.children[3].children[1].children[0].value = df_fx[df_fx['Country']==iso3code]['Annual power required (KWh)'].values[0]
    
    #electricity
    df_fc = dfs_costs['electricity_capex']
    tabs.children[4].children[0].children[0].children[1].value = df_fc[df_fc['Country']==iso3code]['Solar cost (USD/Watt)'].values[0]
    df_fo = dfs_costs['electricity_opex']
    tabs.children[4].children[0].children[1].children[1].value = df_fo[df_fo['Country']==iso3code]['Cost per kWh'].values[0]
    df_fx = dfs_costs['electricity_cstrs']
    tabs.children[4].children[1].children[0].value = df_fx[df_fx['Country']==iso3code]['Power required per school (Watts)'].values[0]
    
def update_tabs(path,iso3code,container):
    global tabs
    
    # create a dictionary to store the dataframes
    dfs_costs = {}

    blob_items = container_client.list_blobs(name_starts_with=path)
    fns = [item['name'] for item in blob_items]
    # iterate over all files in the specified directory
    for filename in fns:
        if filename.endswith(".csv"):  # check if the file is a CSV
            blob_client = blob_service_client.get_blob_client(container=container, blob=filename)
            fnr = blob_client.download_blob().readall()
            fn = io.BytesIO(fnr)
            df = pd.read_csv(fn)  # read the CSV file into a dataframe
            dfs_costs[filename.split('/')[-1][:-4]] = df  # store the dataframe in the dictionary
            
    #fiber
    df_fc = dfs_costs['fiber_capex']
    tabs.children[0].children[0].children[0].children[1].value = df_fc[df_fc['Country']==iso3code]['Setup cost'].values[0]
    tabs.children[0].children[0].children[0].children[2].value = df_fc[df_fc['Country']==iso3code]['Cost per km'].values[0]
    df_fo = dfs_costs['fiber_opex']
    tabs.children[0].children[0].children[1].children[1].value = df_fo[df_fo['Country']==iso3code]['Cost per Mbps/year'].values[0]
    tabs.children[0].children[0].children[1].children[2].value = df_fo[df_fo['Country']==iso3code]['Maintenance yearly'].values[0]
    tabs.children[0].children[0].children[1].children[3].value = df_fo[df_fo['Country']==iso3code]['Transit Mbps/year'].values[0]
    tabs.children[0].children[0].children[1].children[4].value = df_fo[df_fo['Country']==iso3code]['Maintenance per km'].values[0]
    df_fx = dfs_costs['fiber_cstrs']
    tabs.children[0].children[1].children[0].value = df_fx[df_fx['Country']==iso3code]['Annual power required (KWh)'].values[0]
    tabs.children[0].children[1].children[1].value = df_fx[df_fx['Country']==iso3code]['Max length'].values[0]
    
    #cellular
    df_fc = dfs_costs['cell_capex']
    tabs.children[1].children[0].children[0].children[1].value = df_fc[df_fc['Country']==iso3code]['Setup cost'].values[0]
    df_fo = dfs_costs['cell_opex']
    tabs.children[1].children[0].children[1].children[1].value = df_fo[df_fo['Country']==iso3code]['Cost per Mbps/year'].values[0]
    tabs.children[1].children[0].children[1].children[2].value = df_fo[df_fo['Country']==iso3code]['Maintenance yearly'].values[0]
    tabs.children[1].children[0].children[1].children[3].value = df_fo[df_fo['Country']==iso3code]['Transit Mbps/year'].values[0]
    df_fx = dfs_costs['cell_cstrs']
    tabs.children[1].children[1].children[0].value = df_fx[df_fx['Country']==iso3code]['Annual power required (KWh)'].values[0]
    tabs.children[1].children[1].children[1].value = df_fx[df_fx['Country']==iso3code]['Max length'].values[0]
    
    #p2p
    df_fc = dfs_costs['p2p_capex']
    tabs.children[2].children[0].children[0].children[1].value = df_fc[df_fc['Country']==iso3code]['Setup cost - school'].values[0]
    tabs.children[2].children[0].children[0].children[2].value = df_fc[df_fc['Country']==iso3code]['Setup cost - tower'].values[0]
    df_fo = dfs_costs['p2p_opex']
    tabs.children[2].children[0].children[1].children[1].value = df_fo[df_fo['Country']==iso3code]['Cost per Mbps/year'].values[0]
    tabs.children[2].children[0].children[1].children[2].value = df_fo[df_fo['Country']==iso3code]['Maintenance yearly'].values[0]
    tabs.children[2].children[0].children[1].children[3].value = df_fo[df_fo['Country']==iso3code]['Transit Mbps/year'].values[0]
    df_fx = dfs_costs['p2p_cstrs']
    tabs.children[2].children[1].children[0].value = df_fx[df_fx['Country']==iso3code]['Annual power required (KWh)'].values[0]
    tabs.children[2].children[1].children[1].value = df_fx[df_fx['Country']==iso3code]['Max length'].values[0]
    
    #sat
    df_fc = dfs_costs['sat_capex']
    tabs.children[3].children[0].children[0].children[1].value = df_fc[df_fc['Country']==iso3code]['Setup cost'].values[0]
    df_fo = dfs_costs['sat_opex']
    tabs.children[3].children[0].children[1].children[1].value = df_fo[df_fo['Country']==iso3code]['Cost per Mbps/year'].values[0]
    tabs.children[3].children[0].children[1].children[2].value = df_fo[df_fo['Country']==iso3code]['Maintenance yearly'].values[0]
    tabs.children[3].children[0].children[1].children[3].value = df_fo[df_fo['Country']==iso3code]['Transit Mbps/year'].values[0]
    df_fx = dfs_costs['sat_cstrs']
    tabs.children[3].children[1].children[0].value = df_fx[df_fx['Country']==iso3code]['Annual power required (KWh)'].values[0]
    
    #electricity
    df_fc = dfs_costs['electricity_capex']
    tabs.children[4].children[0].children[0].children[1].value = df_fc[df_fc['Country']==iso3code]['Solar cost (USD/Watt)'].values[0]
    df_fo = dfs_costs['electricity_opex']
    tabs.children[4].children[0].children[1].children[1].value = df_fo[df_fo['Country']==iso3code]['Cost per kWh'].values[0]
    df_fx = dfs_costs['electricity_cstrs']
    tabs.children[4].children[1].children[0].value = df_fx[df_fx['Country']==iso3code]['Power required per school (Watts)'].values[0]

def change_disabled_tabs(value):
     #fiber
    tabs.children[0].children[0].children[0].children[1].disabled = value
    tabs.children[0].children[0].children[0].children[2].disabled = value
    tabs.children[0].children[0].children[1].children[1].disabled = value
    tabs.children[0].children[0].children[1].children[2].disabled = value
    tabs.children[0].children[0].children[1].children[3].disabled = value
    tabs.children[0].children[0].children[1].children[4].disabled = value
    tabs.children[0].children[1].children[0].disabled = value
    tabs.children[0].children[1].children[1].disabled = value
    
    #cellular
    tabs.children[1].children[0].children[0].children[1].disabled = value
    tabs.children[1].children[0].children[1].children[1].disabled = value
    tabs.children[1].children[0].children[1].children[2].disabled = value
    tabs.children[1].children[0].children[1].children[3].disabled = value
    tabs.children[1].children[1].children[0].disabled = value
    tabs.children[1].children[1].children[1].disabled = value
    
    #p2p
    tabs.children[2].children[0].children[0].children[1].disabled = value
    tabs.children[2].children[0].children[0].children[2].disabled = value
    tabs.children[2].children[0].children[1].children[1].disabled = value
    tabs.children[2].children[0].children[1].children[2].disabled = value
    tabs.children[2].children[0].children[1].children[3].disabled = value
    tabs.children[2].children[1].children[0].disabled = value
    tabs.children[2].children[1].children[1].disabled = value
    
    #sat
    tabs.children[3].children[0].children[0].children[1].disabled = value
    tabs.children[3].children[0].children[1].children[1].disabled = value
    tabs.children[3].children[0].children[1].children[2].disabled = value
    tabs.children[3].children[0].children[1].children[3].disabled = value
    tabs.children[3].children[1].children[0].disabled = value
    
    #electricity
    tabs.children[4].children[0].children[0].children[1].disabled = value
    tabs.children[4].children[0].children[1].children[1].disabled = value
    tabs.children[4].children[1].children[0].disabled = value

def get_total_fiber_and_num_schools(T,correction_coef):
    total = 0.0
    schools = set()
    for node1,node2,data in T.edges(data=True):
        if node1!='meta' and node2!='meta':
            total += data['w']
            
        if T.nodes[node1]['label']=='school':
            schools.add(node1)
        if T.nodes[node2]['label']=='school':
            schools.add(node2)
            
    return total*correction_coef,len(schools)

def get_fiber_costs(df_schools,T,years_opex,correction_coef,min_bandwidth):
    total_cost = 0
    total_elec_cost = 0
    tf,num_schools = get_total_fiber_and_num_schools(T,correction_coef)
    total_cost += num_schools*tabs.children[0].children[0].children[0].children[1].value
    total_cost += tf*tabs.children[0].children[0].children[0].children[2].value
    total_cost += num_schools*min_bandwidth*years_opex*tabs.children[0].children[0].children[1].children[1].value
    total_cost += num_schools*years_opex*tabs.children[0].children[0].children[1].children[2].value
    total_cost += num_schools*min_bandwidth*years_opex*tabs.children[0].children[0].children[1].children[3].value
    total_cost += tf*years_opex*tabs.children[0].children[0].children[1].children[4].value
    
    for index,row in df_schools.iterrows():
        if row['giga_id_school'] in T.nodes():
            if row['electricity']!='Yes' and row['electricity']!='yes':
                #solar
                total_elec_cost += tabs.children[4].children[0].children[0].children[1].value*tabs.children[4].children[1].children[0].value
            
            total_elec_cost += years_opex*tabs.children[4].children[0].children[1].children[1].value*tabs.children[0].children[1].children[0].value
    
    return total_cost,total_elec_cost
    
    
def get_cell_costs(df_schools,used,years_opex,min_bandwidth):
    total_cost = 0
    total_elec_cost = 0
    for index,row in df_schools.iterrows():
        if row['giga_id_school'] in used:
            if row['electricity']!='Yes' and row['electricity']!='yes':
                #solar
                total_elec_cost += tabs.children[4].children[0].children[0].children[1].value*tabs.children[4].children[1].children[0].value
            
            total_elec_cost += years_opex*tabs.children[4].children[0].children[1].children[1].value*tabs.children[1].children[1].children[0].value
            
            total_cost += tabs.children[1].children[0].children[0].children[1].value
            total_cost += min_bandwidth*years_opex*tabs.children[1].children[0].children[1].children[1].value
            total_cost += years_opex*tabs.children[1].children[0].children[1].children[2].value
            total_cost += min_bandwidth*years_opex*tabs.children[1].children[0].children[1].children[3].value
        
    return total_cost,total_elec_cost
    
    
def get_p2p_costs(df_schools,used,years_opex,min_bandwidth):
    total_cost = 0
    total_elec_cost = 0
    for index,row in df_schools.iterrows():
        if row['giga_id_school'] in used:
            if row['electricity']!='Yes' and row['electricity']!='yes':
                #solar
                total_elec_cost += tabs.children[4].children[0].children[0].children[1].value*tabs.children[4].children[1].children[0].value
            
            total_elec_cost += years_opex*tabs.children[4].children[0].children[1].children[1].value*tabs.children[2].children[1].children[0].value
            
            total_cost += tabs.children[2].children[0].children[0].children[1].value
            total_cost += tabs.children[2].children[0].children[0].children[2].value
            total_cost += min_bandwidth*years_opex*tabs.children[2].children[0].children[1].children[1].value
            total_cost += years_opex*tabs.children[2].children[0].children[1].children[2].value
            total_cost += min_bandwidth*years_opex*tabs.children[2].children[0].children[1].children[3].value
    return total_cost,total_elec_cost
    
def get_vsat_costs(df_schools,used,years_opex,min_bandwidth):
    total_cost = 0
    total_elec_cost = 0
    for index,row in df_schools.iterrows():
        if row['giga_id_school'] in used:
            if row['electricity']!='Yes' and row['electricity']!='yes':
                #solar
                total_elec_cost += tabs.children[4].children[0].children[0].children[1].value*tabs.children[4].children[1].children[0].value
            
            total_elec_cost += years_opex*tabs.children[4].children[0].children[1].children[1].value*tabs.children[3].children[1].children[0].value
            
            total_cost += tabs.children[3].children[0].children[0].children[1].value
            total_cost += min_bandwidth*years_opex*tabs.children[3].children[0].children[1].children[1].value
            total_cost += years_opex*tabs.children[3].children[0].children[1].children[2].value
            total_cost += min_bandwidth*years_opex*tabs.children[3].children[0].children[1].children[3].value
    return total_cost,total_elec_cost

def plot_costs(xs,ys,maxy,title):

    fig, ax = plt.subplots()
    ax.set_facecolor('lightgray')
    ax.grid(color='white')
    ax.set_xticks([0, 5, 10, 15, 20])
    ax.set_xlim(0, 25)
    ax.set_ylim(0, maxy) 
    for tech in xs:
        ax.plot(xs[tech],ys[tech],color=COLORS_TECHS[tech],label=tech)

    # Add legend
    fig.legend(loc="upper center", bbox_to_anchor=(0.5,0.5), bbox_transform=ax.transAxes, fontsize='x-small')
    ax.set_xlabel('Maximum distance between schools connected by fiber, kms')
    plt.title('Cost of schools by technologies ('+title+')', fontsize='medium')
    ax.set_ylabel('Cost (USD)')

    plt.savefig('aux_files/schools_costs_'+title+'.png')
    plt.close(fig)
    
def do_fiber_row(sid,lat,lon,tf,correction_coef,years_opex,min_bandwidth,electricity):
    row = {'giga_id_school':sid,'lat':lat,'lon':lon,'tech':'fiber'}
    
    capex = tabs.children[0].children[0].children[0].children[1].value
    capex += tf*correction_coef*tabs.children[0].children[0].children[0].children[2].value
    opex = min_bandwidth*years_opex*tabs.children[0].children[0].children[1].children[1].value
    opex += years_opex*tabs.children[0].children[0].children[1].children[2].value
    opex += min_bandwidth*years_opex*tabs.children[0].children[0].children[1].children[3].value
    opex += tf*years_opex*tabs.children[0].children[0].children[1].children[4].value
    if electricity!='Yes' and electricity!='yes':
        #solar
        elec_capex = tabs.children[4].children[0].children[0].children[1].value*tabs.children[4].children[1].children[0].value
    else:
        elec_capex = 0.0
            
    elec_opex = years_opex*tabs.children[4].children[0].children[1].children[1].value*tabs.children[0].children[1].children[0].value
    
    row['capex'] = capex
    row['opex'] = opex
    row['electricity_capex'] = elec_capex
    row['electricity_opex'] = elec_opex
    row['total_capex'] = capex + elec_capex
    row['total_opex'] = opex + elec_opex
    row['total_cost'] = capex+opex+elec_capex+elec_opex
    
    return row

def do_cell_row(sid,lat,lon,years_opex,min_bandwidth,electricity):
    row = {'giga_id_school':sid,'lat':lat,'lon':lon,'tech':'cellular'}
    if electricity!='Yes' and electricity!='yes':
        #solar
        elec_capex = tabs.children[4].children[0].children[0].children[1].value*tabs.children[4].children[1].children[0].value
    else:
        elec_capex = 0.0
            
    elec_opex = years_opex*tabs.children[4].children[0].children[1].children[1].value*tabs.children[1].children[1].children[0].value
            
    capex = tabs.children[1].children[0].children[0].children[1].value
    opex = min_bandwidth*years_opex*tabs.children[1].children[0].children[1].children[1].value
    opex += years_opex*tabs.children[1].children[0].children[1].children[2].value
    opex += min_bandwidth*years_opex*tabs.children[1].children[0].children[1].children[3].value
    
    row['capex'] = capex
    row['opex'] = opex
    row['electricity_capex'] = elec_capex
    row['electricity_opex'] = elec_opex
    row['total_capex'] = capex + elec_capex
    row['total_opex'] = opex + elec_opex
    row['total_cost'] = capex+opex+elec_capex+elec_opex
    
    return row

def do_p2p_row(sid,lat,lon,years_opex,min_bandwidth,electricity):
    row = {'giga_id_school':sid,'lat':lat,'lon':lon,'tech':'p2p'}
    if electricity!='Yes' and electricity!='yes':
        #solar
        elec_capex = tabs.children[4].children[0].children[0].children[1].value*tabs.children[4].children[1].children[0].value
    else:
        elec_capex = 0.0
            
    elec_opex = years_opex*tabs.children[4].children[0].children[1].children[1].value*tabs.children[2].children[1].children[0].value
    
    capex = tabs.children[2].children[0].children[0].children[1].value
    capex += tabs.children[2].children[0].children[0].children[2].value
    opex = min_bandwidth*years_opex*tabs.children[2].children[0].children[1].children[1].value
    opex += years_opex*tabs.children[2].children[0].children[1].children[2].value
    opex += min_bandwidth*years_opex*tabs.children[2].children[0].children[1].children[3].value
    
    row['capex'] = capex
    row['opex'] = opex
    row['electricity_capex'] = elec_capex
    row['electricity_opex'] = elec_opex
    row['total_capex'] = capex + elec_capex
    row['total_opex'] = opex + elec_opex
    row['total_cost'] = capex+opex+elec_capex+elec_opex
    
    return row

def do_vsat_row(sid,lat,lon,years_opex,min_bandwidth,electricity):
    row = {'giga_id_school':sid,'lat':lat,'lon':lon,'tech':'vsat'}
    if electricity!='Yes' and electricity!='yes':
        #solar
        elec_capex = tabs.children[4].children[0].children[0].children[1].value*tabs.children[4].children[1].children[0].value
    else:
        elec_capex = 0.0
            
    elec_opex = years_opex*tabs.children[4].children[0].children[1].children[1].value*tabs.children[3].children[1].children[0].value
            
    capex = tabs.children[3].children[0].children[0].children[1].value
    opex = min_bandwidth*years_opex*tabs.children[3].children[0].children[1].children[1].value
    opex += years_opex*tabs.children[3].children[0].children[1].children[2].value
    opex += min_bandwidth*years_opex*tabs.children[3].children[0].children[1].children[3].value
    
    row['capex'] = capex
    row['opex'] = opex
    row['electricity_capex'] = elec_capex
    row['electricity_opex'] = elec_opex
    row['total_capex'] = capex + elec_capex
    row['total_opex'] = opex + elec_opex
    row['total_cost'] = capex+opex+elec_capex+elec_opex
    
    return row

def do_empty_row(sid,lat,lon,years_opex,min_bandwidth,electricity):
    row = {'giga_id_school':sid,'lat':lat,'lon':lon,'tech':'unconnected'}

    row['capex'] = 0.0
    row['opex'] = 0.0
    row['electricity_capex'] = 0.0
    row['electricity_opex'] = 0.0
    row['total_capex'] = 0.0
    row['total_opex'] = 0.0
    row['total_cost'] = 0.0
    
    return row

def calculate_all_school_costs(df_schools,MSTf,cell_used_f, p2p_used_f, vsat_used_f,valid_schools,correction_coef,years_opex,min_bandwidth):
    colnames = ['giga_id_school','lat','lon','tech','capex','opex','electricity_capex','electricity_opex','total_capex','total_opex','total_cost']
    df_school_costs = pd.DataFrame(columns=colnames)
    
    for index,row in df_schools.iterrows():
        sid = row['giga_id_school']
        if sid in valid_schools:
            if sid in MSTf:
                path = list(nx.shortest_path(MSTf, "meta", sid))
                fl = MSTf.edges[path[-2],path[-1]]['w']
                new_row = do_fiber_row(sid,row['lat'],row['lon'],fl,correction_coef,years_opex,min_bandwidth,row['electricity'])
                #df_school_costs = df_school_costs.append(new_row, ignore_index=True)
                df_school_costs = pd.concat([df_school_costs, pd.DataFrame([new_row])], ignore_index=True)
            elif sid in cell_used_f:
                new_row = do_cell_row(sid,row['lat'],row['lon'],years_opex,min_bandwidth,row['electricity'])
                #df_school_costs = df_school_costs.append(new_row, ignore_index=True)
                df_school_costs = pd.concat([df_school_costs, pd.DataFrame([new_row])], ignore_index=True)
            elif sid in p2p_used_f:
                new_row = do_p2p_row(sid,row['lat'],row['lon'],years_opex,min_bandwidth,row['electricity'])
                #df_school_costs = df_school_costs.append(new_row, ignore_index=True)
                df_school_costs = pd.concat([df_school_costs, pd.DataFrame([new_row])], ignore_index=True)
            elif sid in vsat_used_f:
                new_row = do_vsat_row(sid,row['lat'],row['lon'],years_opex,min_bandwidth,row['electricity'])
                #df_school_costs = df_school_costs.append(new_row, ignore_index=True)
                df_school_costs = pd.concat([df_school_costs, pd.DataFrame([new_row])], ignore_index=True)
            else:
                new_row = do_empty_row(sid,row['lat'],row['lon'],years_opex,min_bandwidth,row['electricity'])
                #df_school_costs = df_school_costs.append(new_row, ignore_index=True)
                df_school_costs = pd.concat([df_school_costs, pd.DataFrame([new_row])], ignore_index=True)
                
            
    return df_school_costs 

def plot_school_costs_gradient(gdf,df):
    # Load GeoJSON data
    #gdf = gpd.read_file(fn)

    # Create the plot
    fig, ax = plt.subplots(1, 1)
    gdf.plot(ax=ax,edgecolor='black', facecolor='lightgrey')
    ax.axis('off')
    #plt.title('School connectivity status')
    fig.text(0.5, 0.01, "School costs", ha="center", fontsize=10)
    
    # Create a scatter plot where the color of each point depends on the 'feature' column
    sc = plt.scatter(df['lon'], df['lat'], c=df['total_cost'], cmap='coolwarm', s=2)

    # Add a colorbar
    plt.colorbar(sc, label='Total cost')

    # Save the figure
    plt.savefig('aux_files/school_costs_gradient.png', bbox_inches='tight', pad_inches=0)
    plt.close(fig)
    
def plot_capex_opex(capex,opex):
    N = 3

    ind = np.arange(N)  # the x locations for the groups
    width = 0.5       # the width of the bars: can also be len(x) sequence
    fig, ax = plt.subplots(1, 1)
    
    p1 = ax.bar(ind, capex, width)
    p2 = ax.bar(ind, opex, width, bottom=capex)

    plt.ylabel('Cost (USD)')
    plt.title('Cost distirbution by type')
    plt.xticks(ind, ('Technology','Electricity','Total'))
    plt.legend((p1[0], p2[0]), ('Capex', 'Opex'))
    plt.savefig('aux_files/capex_opex.png', bbox_inches='tight', pad_inches=0)
    plt.close(fig)
    

In [9]:
# Global variables

In [10]:
all_outputs = []
blob_path = ""
iso3code = None
cell_techs = ['LTE'] #for 4G
visible_towers = {}

tab_titles = ['Fiber','Cellular','P2P','Satellite','Electricity']
accordion_titles = ['Costs','Constraints']
tabs = widgets.Tab(children=[create_accordion(accordion_titles,tab_titles[i]) for i in range(len(tab_titles))])


# Set the title of each tab
for i in range(len(tab_titles)):
    tabs.set_title(i, f'{tab_titles[i]}')
    
    
#blob
blob_service_client = BlobServiceClient.from_connection_string(ADLS_CONNECTION_STRING, logger=logger)
container_client = blob_service_client.get_container_client(container=ADLS_CONTAINER)

In [11]:
## Please, add the directory where you mounted blob storage

In [12]:
#wBlob = widgets.Text(
#    value='',
#    placeholder='blob storage',
#    description='String:',
#    disabled=False,
#    layout=widgets.Layout(width='50%')
#)
#outputBlob = widgets.Output()
#all_outputs.append(outputBlob)


#display(wBlob,outputBlob)

In [13]:
filename = 'source/misc/iso3codes.csv'
blob_client = blob_service_client.get_blob_client(container=ADLS_CONTAINER, blob=filename)
fnr = blob_client.download_blob().readall()
fn = io.BytesIO(fnr)
code2country = readCodes(fn)            
#code2country = readCodes('/home/azureuser/cloudfiles/code/Users/jdoturodriguez/data/creditScore/iso3codes.csv')

In [14]:
path = 'source/infra/processed/'
blob_items = container_client.list_blobs(name_starts_with=path)
fns = [item['name'] for item in blob_items]
#avail_codes = [d for d in os.listdir('./') if os.path.isdir(os.path.join('./',d)) and d[0]!='.' and d!='aux_files']
avail_codes = [d.split('/')[-1] for d in fns if d.split('/')[-1] in code2country]
file_options = [('None','None')]
for c in avail_codes:
    file_options.append((code2country[c],c))
file_value = file_options[0][1]

## Please, select the country (and wait for the message to continue)

In [15]:
wCF = widgets.Dropdown(options=file_options,value=file_value,description='Countries')
outputCF = widgets.Output()
display(wCF,outputCF)
all_outputs.append(outputCF)

def on_changeCF(change):
    global iso3code
    global tabs
    with outputCF:
        if change['type'] == 'change' and change['name'] == 'value' and change['new']!='None':
            clear_output(wait=True)
            iso3code = change['new']
            #update_tabs_blob(wBlob.value+'giga/source/infra/costs/',iso3code)
            update_tabs('source/infra/costs/',iso3code,ADLS_CONTAINER)
            sys.stdout.write('Costs updated!')
            sys.stdout.flush()
            
wCF.observe(on_changeCF)

Dropdown(description='Countries', options=(('None', 'None'), ('Benin', 'BEN'), ('Brazil', 'BRA'), ('Guinea', '…

Output()

## Now check the parameters and change them if you need to

### Fiber

In [16]:
wFCC = widgets.IntSlider(
    value=20,
    min=0,
    max=100,
    step=1,
    description='',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d'
)
outputFCC = widgets.Output()
all_outputs.append(outputFCC)

wFCC_HBOX = widgets.HBox([widgets.Label(value="Correction coeficient in %:"), wFCC])
display(wFCC_HBOX,outputFCC)

Output()

In [17]:
checkboxesF = [
    widgets.Checkbox(value=True, description='2'),
    widgets.Checkbox(value=True, description='5'),
    widgets.Checkbox(value=True, description='10'),
    widgets.Checkbox(value=True, description='17')
]

wFML = widgets.GridBox(children=checkboxesF)#, layout=widgets.Layout(grid_template_columns="repeat(3, 200px)"))
outputFML = widgets.Output()
all_outputs.append(outputFML)

wFML_VBOX = widgets.VBox([widgets.Label(value="Maximum connection length(s) in kms:"), wFML])
display(wFML_VBOX,outputFML)

Output()

In [18]:
wFNT = widgets.RadioButtons(
    options=['Only fiber nodes', 'Fiber nodes and connected schools', 'Fiber nodes and connected schools with fiber'],
    value='Only fiber nodes', # Defaults to 'pineapple'
#    layout={'width': 'max-content'}, # If the items' names are long
    description='What entities can initiate a fiber connection?',
    disabled=False
)

outputFNT = widgets.Output()
all_outputs.append(outputFNT)

display(wFNT,outputFNT)

RadioButtons(description='What entities can initiate a fiber connection?', options=('Only fiber nodes', 'Fiber…

Output()

In [19]:
checkboxesFP = [
    widgets.Checkbox(value=True, description='5'),
    widgets.Checkbox(value=True, description='10'),
    widgets.Checkbox(value=True, description='15'),
    widgets.Checkbox(value=True, description='20'),
    widgets.Checkbox(value=True, description='100')
]

wFPML = widgets.GridBox(children=checkboxesFP)#, layout=widgets.Layout(grid_template_columns="repeat(3, 200px)"))
outputFPML = widgets.Output()
all_outputs.append(outputFPML)

wFPML_VBOX = widgets.VBox([widgets.Label(value="Distance brackets (in kms), for plotting purposes:"), wFPML])
display(wFPML_VBOX,outputFPML)

Output()

### Cell towers

In [20]:
wCS = widgets.IntSlider(
    value=6,
    min=0,
    max=12,
    step=1,
    description='',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d'
)
outputCS = widgets.Output()
all_outputs.append(outputCS)

wCS_HBOX = widgets.HBox([widgets.Label(value="Maximum distance to consider the signal strong (in kms):"), wCS])
display(wCS_HBOX,outputCS)

Output()

In [21]:
checkboxesCT = [
    widgets.Checkbox(value=True, description='LTE'),
    widgets.Checkbox(value=False, description='UMTS'),
    widgets.Checkbox(value=False, description='GSM')
]

wCT = widgets.GridBox(children=checkboxesCT)#, layout=widgets.Layout(grid_template_columns="repeat(3, 200px)"))
outputCT = widgets.Output()
all_outputs.append(outputCT)

wCT_VBOX = widgets.VBox([widgets.Label(value="Technology(ies) considered for strong signal to connect schools:"), wCT])
display(wCT_VBOX,outputCT)

Output()

In [22]:
checkboxesC = [
    widgets.Checkbox(value=True, description='2'),
    widgets.Checkbox(value=True, description='5'),
    widgets.Checkbox(value=True, description='10'),
    widgets.Checkbox(value=True, description='15'),
    widgets.Checkbox(value=True, description='30')
]

wCML = widgets.GridBox(children=checkboxesC)#, layout=widgets.Layout(grid_template_columns="repeat(3, 200px)"))
outputCML = widgets.Output()
all_outputs.append(outputCML)

wCML_VBOX = widgets.VBox([widgets.Label(value="Distance brackets (in kms), for plotting purposes:"), wCML])
display(wCML_VBOX,outputCML)

Output()

### P2P

In [23]:
wP2TH = widgets.IntSlider(
    value=35,
    min=0,
    max=100,
    step=1,
    description='',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d'
)
outputP2TH = widgets.Output()
all_outputs.append(outputP2TH)

wP2TH_HBOX = widgets.HBox([widgets.Label(value="Maximum distance for a P2P link, in kms:"), wP2TH])
display(wP2TH_HBOX,outputP2TH)

Output()

In [24]:
wP2AH = widgets.IntSlider(
    value=15,
    min=0,
    max=100,
    step=1,
    description='',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d'
)
outputP2AH = widgets.Output()
all_outputs.append(outputP2AH)

wP2AH_HBOX = widgets.HBox([widgets.Label(value="Height of the antenna at the school, in meters:"), wP2AH])
display(wP2AH_HBOX,outputP2AH)

Output()

In [25]:
checkboxesP2T = [
    widgets.Checkbox(value=True, description='LTE'),
    widgets.Checkbox(value=False, description='UMTS'),
    widgets.Checkbox(value=False, description='GSM')
]

wP2T = widgets.GridBox(children=checkboxesP2T)#, layout=widgets.Layout(grid_template_columns="repeat(3, 200px)"))
outputP2T = widgets.Output()
all_outputs.append(outputP2T)

wP2T_VBOX = widgets.VBox([widgets.Label(value="Technology(ies) considered for a possible P2P link installation:"), wP2T])
display(wP2T_VBOX,outputP2T)

Output()

In [26]:
wP2TW = widgets.IntSlider(
    value=3,
    min=0,
    max=5,
    step=1,
    description='',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d'
)
outputP2TW = widgets.Output()
all_outputs.append(outputP2TW)

wP2TW_VBOX = widgets.VBox([widgets.Label(value="Number of closest cell towers to be included in visibility study:"), wP2TW])
display(wP2TW_VBOX,outputP2TW)

Output()

In [27]:
checkboxesP2P = [
    widgets.Checkbox(value=True, description='5'),
    widgets.Checkbox(value=True, description='10'),
    widgets.Checkbox(value=True, description='15'),
    widgets.Checkbox(value=True, description='20'),
    widgets.Checkbox(value=True, description='100')
]

wP2PML = widgets.GridBox(children=checkboxesP2P)#, layout=widgets.Layout(grid_template_columns="repeat(3, 200px)"))
outputP2PML = widgets.Output()
all_outputs.append(outputP2PML)

wP2PML_VBOX = widgets.VBox([widgets.Label(value="Distance brackets (in kms) for P2P nodes, for plotting purposes:"), wP2PML])
display(wP2PML_VBOX,outputP2PML)

Output()

In [28]:
checkboxesVB = [
    widgets.Checkbox(value=True, description='1'),
    widgets.Checkbox(value=True, description='2'),
    widgets.Checkbox(value=True, description='3'),
    widgets.Checkbox(value=True, description='5'),
    widgets.Checkbox(value=True, description='35')
]

wVB = widgets.GridBox(children=checkboxesVB)#, layout=widgets.Layout(grid_template_columns="repeat(3, 200px)"))
outputVB = widgets.Output()
all_outputs.append(outputVB)

wVB_VBOX = widgets.VBox([widgets.Label(value="Distance brackets (in kms) for visible towers, for plotting purposes:"), wVB])
display(wVB_VBOX,outputVB)

Output()

## General

In [29]:
wSCT = widgets.RadioButtons(
    options=['Only unconnected schools', 'Unconnected and unknown schools', 'All schools'],
    value='Unconnected and unknown schools', # Defaults to 'pineapple'
#    layout={'width': 'max-content'}, # If the items' names are long
    description='What schools do you want to connect?',
    disabled=False
)

outputSCT = widgets.Output()
all_outputs.append(outputSCT)

display(wSCT,outputSCT)

RadioButtons(description='What schools do you want to connect?', index=1, options=('Only unconnected schools',…

Output()

In [30]:
wSATa = widgets.Checkbox(
    value=True,
    description='Satellite Coverage available',
    disabled=False,
    indent=False
)
outputSATa = widgets.Output()
all_outputs.append(outputSATa)

display(wSATa,outputSATa)

Checkbox(value=True, description='Satellite Coverage available', indent=False)

Output()

In [31]:
wYO = widgets.IntSlider(
    value=5,
    min=0,
    max=10,
    step=1,
    description='',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d'
)
outputYO = widgets.Output()
all_outputs.append(outputYO)

wYO_HBOX = widgets.HBox([widgets.Label(value="Number of opex years to consider in cost study:"), wYO])
display(wYO_HBOX,outputYO)

Output()

In [32]:
wMB = widgets.IntSlider(
    value=20,
    min=0,
    max=100,
    step=1,
    description='',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d'
)
outputMB = widgets.Output()
all_outputs.append(outputMB)

wMB_HBOX = widgets.HBox([widgets.Label(value="Minimum bandwidth for all schools, in Mbps:"), wMB])
display(wMB_HBOX,outputMB)

Output()

## Costs and Constraints

In [33]:



    
display(tabs)

## Generate Report
### Click the button below, when it is finsihed a link will appear for you to download

In [38]:
buttonR = widgets.Button(description="Generate Report")
outputR = widgets.Output()
progress = widgets.IntProgress(value=0, min=0, max=100, step=1, description='Generating:', bar_style='', orientation='horizontal')
progress_label = widgets.Label(value="0%")
progress_HBOX = widgets.HBox([progress,progress_label])
outputPHBOX = widgets.Output()
display(buttonR,outputR)
all_outputs.append(outputR)
all_outputs.append(outputPHBOX)

    
def on_button_clickedR(b):
    
    with outputR:
        clear_output(wait=False)
        if wCF.value != 'None':
            display(progress_HBOX,outputPHBOX)
            
            #copy files and dirs for report
            plots_directory ='./aux_files'
            if not os.path.exists(plots_directory):
                # If it doesn't exist, create it
                os.makedirs(plots_directory)
                
            local_logo_path = './aux_files/giga_logo.png'
            blob_client = blob_service_client.get_blob_client(container=ADLS_CONTAINER, blob='source/infra/reports/aux_files/giga_logo.png')
            with open(local_logo_path, "wb") as download_file:
                download_file.write(blob_client.download_blob().readall())
                
            report_directory='./'+iso3code
            if not os.path.exists(report_directory):
                # If it doesn't exist, create it
                os.makedirs(report_directory)
                
            buttonR.disabled = True
            #disable widgets
            #wBlob.disabled = True
            wCF.disabled = True
            wFCC.disabled = True
            for i in range(0, len(checkboxesF)):
                checkboxesF[i].disabled = True
            for i in range(0, len(checkboxesFP)):
                checkboxesFP[i].disabled = True
            wCS.disabled = True
            for i in range(0, len(checkboxesC)):
                checkboxesC[i].disabled = True
            for i in range(0, len(checkboxesVB)):
                checkboxesVB[i].disabled = True
                
            wP2TH.disabled = True
            wP2AH.disabled = True
            
            for i in range(0, len(checkboxesCT)):
                checkboxesCT[i].disabled = True
                
            wP2TW.disabled = True
            
            wYO.disabled = True
            wMB.disabled = True
            
            wSCT.disabled = True
            wFNT.disabled = True
            
            for i in range(0, len(checkboxesP2P)):
                checkboxesP2P[i].disabled = True
            for i in range(0, len(checkboxesP2T)):
                checkboxesP2T[i].disabled = True
                
            wSATa.disabled = True
            change_disabled_tabs(True)
                
            #read params
            #print('read params')
            
            #blob_path = wBlob.value
            correction_coef = 1.0 + wFCC.value/100.0
            fiber_thresholds = []
            for i in range(0, len(checkboxesF)):
                if checkboxesF[i].value == True:
                    fiber_thresholds.append(int(checkboxesF[i].description))
            fiber_plot_brackets = []
            for i in range(0, len(checkboxesFP)):
                if checkboxesFP[i].value == True:
                    fiber_plot_brackets.append(int(checkboxesFP[i].description))
            cell_plot_brackets = []
            for i in range(0, len(checkboxesC)):
                if checkboxesC[i].value == True:
                    cell_plot_brackets.append(int(checkboxesC[i].description))
            visibility_brackets = []
            for i in range(0, len(checkboxesVB)):
                if checkboxesVB[i].value == True:
                    visibility_brackets.append(int(checkboxesVB[i].description))
                    
            cell_techs = []
            for i in range(0, len(checkboxesCT)):
                if checkboxesCT[i].value == True:
                    cell_techs.append(checkboxesCT[i].description)
                    
            cell_strong = wCS.value
            p2p_threshold = wP2TH.value
            antenna_height = wP2AH.value
            num_towers_output = wP2TW.value
            
            p2p_techs = []
            for i in range(0, len(checkboxesP2T)):
                if checkboxesP2T[i].value == True:
                    p2p_techs.append(checkboxesP2T[i].description)
            p2p_plot_brackets = []
            for i in range(0, len(checkboxesP2P)):
                if checkboxesP2P[i].value == True:
                    p2p_plot_brackets.append(int(checkboxesP2P[i].description))
                    
            schools_to_connect = wSCT.value
            fiber_node_types = wFNT.value
            years_opex = wYO.value
            min_bandwidth = wMB.value
            sat_avail = wSATa.value
            
            #read files
            #print('read file')
            ###remove this soon!!!!
            if iso3code=='BRA':
                #df_schools = pd.read_csv('/home/azureuser/cloudfiles/code/Users/jdoturodriguez/notebooks/creditScore/BRA_with_admin1_and_census.csv')
                filename = "source/infra/processed/"+iso3code+"/"+iso3code+"_with_admin1_and_census.csv"
            else:
                filename = "gold/school_data/"+iso3code+"_school_geolocation_coverage_master.csv"
            blob_client = blob_service_client.get_blob_client(container=ADLS_CONTAINER, blob=filename)
            if not blob_client.exists():
                sys.stdout.write('Missing school file!')
                sys.stdout.flush()
                return
            fnr = blob_client.download_blob().readall()
            fn = io.BytesIO(fnr)
            df_schools = pd.read_csv(fn)
            #df_schools = pd.read_csv(blob_path+"giga/gold/school_data/"+iso3code+"_school_geolocation_coverage_master.csv")
            filename = "source/infra/processed/"+iso3code+"/"+iso3code+"_cell.csv"
            blob_client = blob_service_client.get_blob_client(container=ADLS_CONTAINER, blob=filename)
            if not blob_client.exists():
                sys.stdout.write('Missing cellular file!')
                sys.stdout.flush()
                return
            fnr = blob_client.download_blob().readall()
            fn = io.BytesIO(fnr)
            df_cell = pd.read_csv(fn)
            #df_cell = pd.read_csv(iso3code+"/"+iso3code+"_cell.csv")
            filename = "source/infra/processed/"+iso3code+"/"+iso3code+"_fiber.csv"
            blob_client = blob_service_client.get_blob_client(container=ADLS_CONTAINER, blob=filename)
            if not blob_client.exists():
                sys.stdout.write('Missing fiber file!')
                sys.stdout.flush()
                return
            fnr = blob_client.download_blob().readall()
            fn = io.BytesIO(fnr)
            df_fiber = pd.read_csv(fn)
            #df_fiber = pd.read_csv(iso3code+"/"+iso3code+"_fiber.csv")
            filename = "source/infra/processed/"+iso3code+"/"+iso3code+"_p2p.csv"
            blob_client = blob_service_client.get_blob_client(container=ADLS_CONTAINER, blob=filename)
            if not blob_client.exists():
                df_p2p = pd.DataFrame()
            else:
                fnr = blob_client.download_blob().readall()
                fn = io.BytesIO(fnr)
                df_p2p = pd.read_csv(fn)
            #df_p2p = pd.read_csv(iso3code+"/"+iso3code+"_p2p.csv")
            
            filename = "source/infra/processed/"+iso3code+"/"+iso3code+"_world-administrative-boundaries.geojson"
            blob_client = blob_service_client.get_blob_client(container=ADLS_CONTAINER, blob=filename)
            if not blob_client.exists():
                sys.stdout.write('Missing country boundary geojson!')
                sys.stdout.flush()
                return
            fnr = blob_client.download_blob().readall()
            fn = io.BytesIO(fnr)
            gdf = gpd.read_file(fn)
            
            ###read caches###
            #print('read caches')
            progress.value = 5
            progress_label.value = "5%"
            #fiber
            filename = "source/infra/processed/"+iso3code+"/"+iso3code+"_fnode_dists.pkl"
            blob_client = blob_service_client.get_blob_client(container=ADLS_CONTAINER, blob=filename)
            if not blob_client.exists():
                sys.stdout.write('Missing fiber cache!')
                sys.stdout.flush()
                return
            fnr = blob_client.download_blob().readall()
            fn = io.BytesIO(fnr)
            #with open(iso3code+"/"+iso3code+"_fnode_dists.pkl", "rb") as f:
            fnode_dists = pickle.load(fn)
    
            #cell
            progress.value = 6
            progress_label.value = "6%"
            filename = "source/infra/processed/"+iso3code+"/"+iso3code+"_cell_dists.pkl"
            blob_client = blob_service_client.get_blob_client(container=ADLS_CONTAINER, blob=filename)
            if not blob_client.exists():
                sys.stdout.write('Missing cellular cache!')
                sys.stdout.flush()
                return
            fnr = blob_client.download_blob().readall()
            fn = io.BytesIO(fnr)
            #with open(iso3code+"/"+iso3code+"_cell_dists.pkl", "rb") as f:
            #with open(fn,'rb') as f:
            cell_dists = pickle.load(fn)
    
            #p2p
            progress.value = 7
            progress_label.value = "7%"
            filename = "source/infra/processed/"+iso3code+"/"+iso3code+"_p2p_dists.pkl"
            blob_client = blob_service_client.get_blob_client(container=ADLS_CONTAINER, blob=filename)
            if not df_p2p.empty:
                if not blob_client.exists():
                    sys.stdout.write('Missing p2p cache!')
                    sys.stdout.flush()
                    return
                fnr = blob_client.download_blob().readall()
                fn = io.BytesIO(fnr)
            #with open(iso3code+"/"+iso3code+"_p2p_dists.pkl", "rb") as f:
            #with open(fn,'rb') as f:
                p2p_dists = pickle.load(fn)
    
            #schools
            progress.value = 8
            progress_label.value = "8%"
            filename = "source/infra/processed/"+iso3code+"/"+iso3code+"_school_dists.pkl"
            blob_client = blob_service_client.get_blob_client(container=ADLS_CONTAINER, blob=filename)
            if not blob_client.exists():
                sys.stdout.write('Missing school cache!')
                sys.stdout.flush()
                return
            fnr = blob_client.download_blob().readall()
            fn = io.BytesIO(fnr)
            #with open(iso3code+"/"+iso3code+"_school_dists.pkl", "rb") as f:
            #with open(fn,'rb') as f:
            school_dists = pickle.load(fn)
            
            #####Generate Report#####
            #print('generate report')
            progress.value = 10
            progress_label.value = "10%"
            #Doc
            geometry_options = {"tmargin":"1cm","lmargin":"1cm","margin":"1cm"}
            
            doc = Document(geometry_options=geometry_options,documentclass="beamer")
            doc.preamble.append(Command(NoEscape(r'usepackage[absolute,overlay]{textpos}')))
            #doc.packages.append(Package('tikz'))
            doc.preamble.append(Command('usetheme', 'CambridgeUS'))
            doc.preamble.append(Command('usecolortheme', 'whale'))
            doc.preamble.append(Command(NoEscape(r'setbeamercolor*{palette tertiary}{use=structure,fg=white,bg=black}')))
            doc.preamble.append(Command(NoEscape(r'setbeamercolor*{palette quaternary}{fg=white,bg=black}')))
            doc.preamble.append(Command(NoEscape(r'setbeamercolor{frametitle}{bg=structure!75,fg=white}')))
            #doc.preamble.append(Command(NoEscape(r'usetikzlibrary{shapes.callouts}')))
            #doc.preamble.append(Command(NoEscape(r'usetikzlibrary{decorations.text}')))
            
            #Title page
            with doc.create(Section('MVP')):  
                doc.preamble.append(Command('title', 'Infrastructure Analysis'))
                doc.preamble.append(Command('subtitle', f"{code2country[iso3code]}"))
                doc.preamble.append(Command('author', 'Giga'))
                doc.preamble.append(Command('date', NoEscape(r'\today')))
                doc.append(NoEscape(r'\maketitle'))
                
            
            #Data Assessment
            
            plot_country_map(gdf)
            with doc.create(Section('Data Assessment')):
                doc.append(VerticalSpace("5pt")) 
                doc.append(NoEscape(r'{\Large Number of records:}'))
                doc.append(NoEscape(r'\\'))  # Start a new line
                with doc.create(MiniPage(width=r"0.35\textwidth")):
                    with doc.create(Itemize()) as itemize:
                        itemize.add_item(f"{len(df_schools)} schools")
                        itemize.add_item(f"{len(df_fiber)} fiber nodes")
                        itemize.add_item(f"{len(df_cell)} cell towers")
                        itemize.add_item(f"{len(df_p2p)} microwave nodes")
                with doc.create(MiniPage(width=r"0.60\textwidth")):
                    with doc.create(Figure(position='h!')) as fig:
                        fig.add_image('../aux_files/country_map.png', width=NoEscape(r'0.4\textwidth'))
                    
                #doc.append(NoEscape(r"""
                #\begin{tikzpicture}[remember picture, overlay]
                #\node[anchor=east,xshift=-2cm] at (current page.east)
                #{\includegraphics[width=.25\textwidth]{../aux_files/country_map.png}};
                #\end{tikzpicture}
                #"""))
                    
                doc.append(NoEscape(r"""
                \begin{textblock*}{3cm}(10.1cm,8.8cm)  % {block width} (coords)
                \includegraphics[width=.9\textwidth]{../aux_files/giga_logo.png}
                \end{textblock*}
                """))
                doc.append(NewPage())
                
            #Data Quality - Schools
            progress.value = 14
            progress_label.value = "14%"
            with doc.create(Section('Data Quality')):
                doc.append(VerticalSpace("5pt")) 
                doc.append(NoEscape(r'{\Large Schools:}'))
                doc.append(NoEscape(r'\\'))  # Start a new line
                doc.append(VerticalSpace("2pt")) 
                doc.append(NoEscape(r"Connectivity status: \hspace{4cm} Electricity status:"))
                num_nan_conn = df_schools['connectivity'].isna().sum()
                plot_piechart_nans(df_schools,'connectivity')
                num_nan_ele = df_schools['electricity'].isna().sum()
                plot_piechart_nans(df_schools,'electricity')
                with doc.create(Figure(position='h!')) as fig:
                    with doc.create(SubFigure(position='b', width=NoEscape(r'0.45\textwidth'))) as pie_chart1:
                        pie_chart1.add_image('../aux_files/connectivity_nan_pie.png', width=NoEscape(r'\textwidth'))
                        pie_chart1.add_caption(f"Unknown for {num_nan_conn} schools")
                    doc.append(NoEscape(r'\hspace{1cm}'))  # To add horizontal space between the figures
                    with doc.create(SubFigure(position='b', width=NoEscape(r'0.45\textwidth'))) as pie_chart2:
                        pie_chart2.add_image('../aux_files/electricity_nan_pie.png', width=NoEscape(r'\textwidth'))
                        pie_chart2.add_caption(f"Unknown for {num_nan_ele} schools")

                    
                doc.append(NoEscape(r"""
                \begin{textblock*}{3cm}(10.1cm,8.8cm)  % {block width} (coords)
                \includegraphics[width=.9\textwidth]{../aux_files/giga_logo.png}
                \end{textblock*}
                """))
                
                doc.append(NewPage())
                
            #Data Quality - Cell towers
            progress.value = 18
            progress_label.value = "18%"
            if not df_cell['backhaul_throuput_mbps'].isnull().all() and not df_cell['backhaul_type'].isnull().all():
                with doc.create(Section('Data Quality II')):
                    doc.append(VerticalSpace("5pt")) 
                    doc.append(NoEscape(r'{\Large Cell towers:}'))
                    doc.append(NoEscape(r'\\'))  # Start a new line
                    doc.append(VerticalSpace("2pt")) 
                    doc.append(NoEscape(r"Backhaul throuput: \hspace{4cm} Backhaul distribution:"))
                
                    num_nan_bk = df_cell['backhaul_throuput_mbps'].isna().sum()
                    plot_piechart_nans(df_cell,'backhaul_throuput_mbps')
                    if (df_cell['backhaul_type']=='FH').any():
                        plot_piechart_counts_with_dict(df_cell,'backhaul_type',{'FH':'Micro Wave'})
                    else:
                        plot_piechart_counts_with_dict(df_cell,'backhaul_type',{})
                    with doc.create(Figure(position='h!')) as fig:
                        with doc.create(SubFigure(position='b', width=NoEscape(r'0.45\textwidth'))) as pie_chart1:
                            pie_chart1.add_image('../aux_files/backhaul_throuput_mbps_nan_pie.png', width=NoEscape(r'\textwidth'))
                            pie_chart1.add_caption(f"Unknown for {num_nan_bk} cell towers")
                        doc.append(NoEscape(r'\hspace{1cm}'))  # To add horizontal space between the figures
                        with doc.create(SubFigure(position='b', width=NoEscape(r'0.45\textwidth'))) as pie_chart2:
                            pie_chart2.add_image('../aux_files/backhaul_type_pie.png', width=NoEscape(r'\textwidth'))
                            pie_chart2.add_caption("Different backhaul technologies")

                    
                doc.append(NoEscape(r"""
                \begin{textblock*}{3cm}(10.1cm,8.8cm)  % {block width} (coords)
                \includegraphics[width=.9\textwidth]{../aux_files/giga_logo.png}
                \end{textblock*}
                """))
                doc.append(NewPage())
                
            #State of Infrastructure - Connectivity 
            progress.value = 22
            progress_label.value = "22%"
            map_connectivity_schools(gdf,df_schools)
            with doc.create(Section('State of Infrastructure')):
                doc.append(VerticalSpace("5pt")) 
                doc.append(NoEscape(r'{\Large Connectivity:}'))
                with doc.create(Figure(position='h!')) as fig:
                    fig.add_image('../aux_files/connectivity_map.png', width=NoEscape(r'0.4\textwidth'))
                    #fig.add_caption('School connectivity status')
                
                doc.append(NoEscape(r"""
                \begin{textblock*}{3cm}(10.1cm,8.8cm)  % {block width} (coords)
                \includegraphics[width=.9\textwidth]{../aux_files/giga_logo.png}
                \end{textblock*}
                """))
                doc.append(NewPage())
                
                
            #State of Infrastructure - Fiber
            progress.value = 26
            progress_label.value = "26%"
            plot_map_distances(gdf,df_schools,fnode_dists,fiber_plot_brackets,'fiber')
            with doc.create(Section('State of Infrastructure II')):
                doc.append(VerticalSpace("5pt")) 
                doc.append(NoEscape(r'{\Large Fiber:}'))
                with doc.create(Figure(position='h!')) as fig:
                    fig.add_image('../aux_files/fiber_dists_map.png', width=NoEscape(r'0.4\textwidth'))
                
                doc.append(NoEscape(r"""
                \begin{textblock*}{3cm}(10.1cm,8.8cm)  % {block width} (coords)
                \includegraphics[width=.9\textwidth]{../aux_files/giga_logo.png}
                \end{textblock*}
                """))
                doc.append(NewPage())
                
            #State of Infrastructure - Fiber II
            progress.value = 30  
            progress_label.value = "30%"
            data = []
            for k in fnode_dists:
                if len(fnode_dists[k])==0:
                    data.append(fiber_plot_brackets[-1]*1.2)
                else:
                    data.append(min(fnode_dists[k].values()))
            
            pctg_th_fiber = sum([1 for d in data if d<=fiber_dist_plot_threshold])/len(df_schools)*100
            plot_dist_distributions(data,fiber_dist_plot_threshold,pctg_th_fiber,'fiber node')
            with doc.create(Section('State of Infrastructure III')):
                doc.append(VerticalSpace("5pt")) 
                doc.append(NoEscape(r'{\Large Fiber:}'))
                doc.append(NoEscape(r'\\'))  # Start a new line
                doc.append(VerticalSpace("2pt")) 
                with doc.create(MiniPage(width=r"0.3\textwidth")):
                    doc.append("At least ")
                    doc.append(bold(str(int(pctg_th_fiber))+"% "))
                    doc.append(" of schools are located within "+str(fiber_dist_plot_threshold)+" kms to the closest fiber node")

                with doc.create(MiniPage(width=r"0.68\textwidth")):
                    with doc.create(Figure(position='h!')) as fig:
                        fig.add_image("../aux_files/dist_distributions_fiber.png", width=NoEscape(r"0.9\textwidth"))

    
                doc.append(NoEscape(r"""
                \begin{textblock*}{3cm}(10.1cm,8.8cm)  % {block width} (coords)
                \includegraphics[width=.9\textwidth]{../aux_files/giga_logo.png}
                \end{textblock*}
                """))
                doc.append(NewPage())
            
            #State of Infrastructure - Cell towers
            progress.value = 34
            progress_label.value = "34%"
            plot_map_distances(gdf,df_schools,cell_dists,cell_plot_brackets,'cell')
            with doc.create(Section('State of Infrastructure IV')):
                doc.append(VerticalSpace("5pt")) 
                doc.append(NoEscape(r'{\Large Cell towers:}'))
                with doc.create(Figure(position='h!')) as fig:
                    fig.add_image('../aux_files/cell_dists_map.png', width=NoEscape(r'0.4\textwidth'))
                
                doc.append(NoEscape(r"""
                \begin{textblock*}{3cm}(10.1cm,8.8cm)  % {block width} (coords)
                \includegraphics[width=.9\textwidth]{../aux_files/giga_logo.png}
                \end{textblock*}
                """))
                doc.append(NewPage())
                
                
            #State of Infrastructure - Cell towers II
            progress.value = 38
            progress_label.value = "38%"
            data = []
            for k in cell_dists:
                if len(cell_dists[k])==0:
                    data.append(cell_plot_brackets[-1]*1.2)
                else:
                    data.append(min(cell_dists[k].values()))
            pctg_th_cell = sum([1 for d in data if d<=cell_dist_plot_threshold])/len(df_schools)*100
            counts_4g_th = less_than_dists_with_condition(cell_dists,cell_dist_plot_threshold,df_cell,'tower_id','technologies','LTE')
            pctg_th_4g = counts_4g_th/len(df_schools)*100 
            plot_dist_distributions(data,cell_dist_plot_threshold,pctg_th_cell,'cell tower')
            with doc.create(Section('State of Infrastructure V')):
                doc.append(VerticalSpace("5pt")) 
                doc.append(NoEscape(r'{\Large Cell Tower:}'))
                doc.append(NoEscape(r'\\'))  # Start a new line
                doc.append(VerticalSpace("2pt")) 
                with doc.create(MiniPage(width=r"0.3\textwidth")):
                    doc.append("At least ")
                    doc.append(bold(str(int(pctg_th_cell))+"% "))
                    doc.append(" of schools are located within "+str(cell_dist_plot_threshold)+" kms to the closest cell tower")
                    doc.append(NoEscape(r'\\'))  # Start a new line
                    doc.append(NoEscape(r'\\'))  # Start a new line
                    doc.append(bold(str(int(pctg_th_4g))+"% "))
                    doc.append(" of schools are located within "+str(cell_dist_plot_threshold)+" kms to the closest 4G cell tower")
                with doc.create(MiniPage(width=r"0.68\textwidth")):
                    with doc.create(Figure(position='h!')) as fig:
                        fig.add_image("../aux_files/dist_distributions_cell.png", width=NoEscape(r"0.9\textwidth"))
    
                doc.append(NoEscape(r"""
                \begin{textblock*}{3cm}(10.1cm,8.8cm)  % {block width} (coords)
                \includegraphics[width=.9\textwidth]{../aux_files/giga_logo.png}
                \end{textblock*}
                """))
                doc.append(NewPage())
            
            #State of Infrastructure - Cell strength
            progress.value = 42
            progress_label.value = "42%"
            plot_signal_strength(gdf,df_cell,cell_techs)
            counts_strong_th = less_than_dists_with_condition(cell_dists,cell_strong,df_cell,'tower_id','technologies','LTE')
            pctg_th_strong = counts_strong_th/len(df_schools)*100
            with doc.create(Section('State of Infrastructure VI')):
                doc.append(VerticalSpace("5pt")) 
                doc.append(NoEscape(r'{\Large 4G Mobile coverage:}'))
                doc.append(NoEscape(r'\\'))  # Start a new line
                doc.append(VerticalSpace("2pt")) 
                with doc.create(MiniPage(width=r"0.3\textwidth")):
                    doc.append("Around ")
                    doc.append(bold(str(int(pctg_th_strong))+"% "))
                    doc.append(" of schools are covered by a strong 4G network")
                with doc.create(MiniPage(width=r"0.68\textwidth")):
                    with doc.create(Figure(position='h!')) as fig:
                        fig.add_image("../aux_files/cell_strength.png", width=NoEscape(r"0.8\textwidth"))

    
                doc.append(NoEscape(r"""
                \begin{textblock*}{3cm}(10.1cm,8.8cm)  % {block width} (coords)
                \includegraphics[width=.9\textwidth]{../aux_files/giga_logo.png}
                \end{textblock*}
                """))
                doc.append(NewPage())
            
            if not df_p2p.empty:
                #State of Infrastructure - P2P
                progress.value = 46
                progress_label.value = "46%"
                plot_map_distances(gdf,df_schools,p2p_dists,p2p_plot_brackets,'p2p')
                with doc.create(Section('State of Infrastructure VII')):
                    doc.append(VerticalSpace("5pt")) 
                    doc.append(NoEscape(r'{\Large P2P:}'))
                    with doc.create(Figure(position='h!')) as fig:
                        fig.add_image('../aux_files/p2p_dists_map.png', width=NoEscape(r'0.4\textwidth'))
                
                    doc.append(NoEscape(r"""
                    \begin{textblock*}{3cm}(10.1cm,8.8cm)  % {block width} (coords)
                    \includegraphics[width=.9\textwidth]{../aux_files/giga_logo.png}
                    \end{textblock*}
                    """))
                    doc.append(NewPage())
                
                #State of Infrastructure - P2P II
                progress.value = 50  
                progress_label.value = "50%"
                data = []
                for k in p2p_dists:
                    if len(p2p_dists[k])==0:
                        data.append(p2p_plot_brackets[-1]*1.2)
                    else:
                        data.append(min(p2p_dists[k].values()))
                pctg_th_p2p = sum([1 for d in data if d<=p2p_dist_plot_threshold])/len(df_schools)*100
                plot_dist_distributions(data,p2p_dist_plot_threshold,pctg_th_p2p,'microwave node')
                with doc.create(Section('State of Infrastructure VIII')):
                    doc.append(VerticalSpace("5pt")) 
                    doc.append(NoEscape(r'{\Large P2P:}'))
                    doc.append(NoEscape(r'\\'))  # Start a new line
                    doc.append(VerticalSpace("2pt")) 
                    with doc.create(MiniPage(width=r"0.3\textwidth")):
                        doc.append("At least ")
                        doc.append(bold(str(int(pctg_th_p2p))+"% "))
                        doc.append(" of schools are located within "+str(p2p_dist_plot_threshold)+" kms to the closest microwave node")

                    with doc.create(MiniPage(width=r"0.68\textwidth")):
                        with doc.create(Figure(position='h!')) as fig:
                            fig.add_image("../aux_files/dist_distributions_microwave.png", width=NoEscape(r"0.9\textwidth"))

    
                    doc.append(NoEscape(r"""
                    \begin{textblock*}{3cm}(10.1cm,8.8cm)  % {block width} (coords)
                    \includegraphics[width=.9\textwidth]{../aux_files/giga_logo.png}
                    \end{textblock*}
                    """))
                    doc.append(NewPage())
            
            #Visbility Analysis
            progress.value = 54
            progress_label.value = "54%"
            path = "source/infra/processed/"+iso3code+"/"
            blob_items = container_client.list_blobs(name_starts_with=path)
            fns = [item['name'] for item in blob_items]
            files = [f for f in fns if f[-3:]=='pkl' and 'elevation_profile_list' in f]
            chosen_f = None
            for f in files:
                words = f.split('_')
                fth = int(words[4][2:])
                ftechs = words[5][2:]
                fnt = int(words[6][2:-4])
                if fth==p2p_threshold:
                    if num_towers_output<=fnt:
                        if len(p2p_techs)==1:
                            if p2p_techs[0]==ftechs:
                                chosen_f = f
                                break
                        elif len(p2p_techs)==3:
                            if ftechs=='ALL':
                                chosen_f = f
                                break
                        else:
                            if p2p_techs[0] in ftechs and p2p_techs[1] in ftechs:
                                chosen_f = f
                                break 
                                
            if chosen_f!=None:
                words = chosen_f.split('/')[-1].split('_')
                coords_f = path+iso3code+'_cell_coordinates_'+words[4]+'_'+words[5]+'_'+words[6]
                blob_client = blob_service_client.get_blob_client(container=ADLS_CONTAINER, blob=coords_f)
                fnr = blob_client.download_blob().readall()
                fn = io.BytesIO(fnr)            
                cell_coordinates = pickle.load(fn)

                blob_client = blob_service_client.get_blob_client(container=ADLS_CONTAINER, blob=chosen_f)
                fnr = blob_client.download_blob().readall()
                fn = io.BytesIO(fnr)            
                elevation_profile_list = pickle.load(fn)
            
                coordinates = cell_coordinates['coordinates']
                school_in_coordinates = cell_coordinates['school_in_coordinates']
                tower_in_coordinates = cell_coordinates['tower_in_coordinates']
                
                for i in range(len(elevation_profile_list)):
                    profile = elevation_profile_list[i]
                    school_id = school_in_coordinates[i]
                    tower_id = tower_in_coordinates[i]
                    tower_height = df_cell[df_cell['tower_id']==tower_id].iloc[0]['height']
                    if pd.isnull(tower_height):
                        tower_height = DEFAULT_TOWER_HEIGHT
                    if school_id not in visible_towers:
                        visible_towers[school_id] = []
                    if len(visible_towers[school_id])<num_towers_output:
                        x,y,z = profile[0]
                        profile[0] = (x,y,z+antenna_height)
                        x,y,z = profile[-1]
                        profile[-1] = (x,y,z+tower_height)
                        for j in range(1,len(profile)-1):
                            x,y,z = profile[j]
                            profile[j] = (x,y,z+los_buffer)
                        #print(i)
                        #print(profile)
                        #print(tower_height)
                        if is_visibleActual(profile):
                            visible_towers[school_id].append((tower_id,cell_dists[school_id][tower_id]))
                            
                plot_visibility_map(gdf,visible_towers,df_schools,df_cell,visibility_brackets)
                
                with doc.create(Section('Visibility Analysis')):
                    doc.append(VerticalSpace("5pt")) 
                    doc.append(NoEscape(r'{\Large Closest visible towers:}'))
   
                    with doc.create(Figure(position='h!')) as fig:
                        fig.add_image("../aux_files/visibility.png", width=NoEscape(r"0.5\textwidth"))

    
                    doc.append(NoEscape(r"""
                    \begin{textblock*}{3cm}(10.1cm,8.8cm)  % {block width} (coords)
                    \includegraphics[width=.9\textwidth]{../aux_files/giga_logo.png}
                    \end{textblock*}
                    """))
                    doc.append(NewPage())
            
                #State of Infrastructure - Visibility Analysis II
                progress.value = 58
                progress_label.value = "58%"
                data = []
                for key in visible_towers:
                    if len(visible_towers[key])>0:
                        sorted_pairs = sorted(visible_towers[key], key=lambda pair: pair[1])
                        tower_id, d = sorted_pairs[0]
                        data.append(d)
                plot_visibility_distributions(data)
                avg_visible = get_avg_visible(visible_towers)
                with doc.create(Section('Visibility Analysis II')):
                    doc.append(VerticalSpace("5pt")) 
                    doc.append(NoEscape(r'{\Large Visibility statistics:}'))
                    doc.append(NoEscape(r'\\'))  # Start a new line
                    doc.append(VerticalSpace("2pt")) 
                    with doc.create(MiniPage(width=r"0.3\textwidth")):
                        doc.append("Percentage of schools visible from at least")
                        with doc.create(Itemize()) as itemize:
                            for i in range(1,num_towers_output+1):
                                num_visible = count_lists_of_at_least_size(visible_towers, i)
                                pctg_visible = num_visible/len(df_schools)*100
                                itemize.add_item(f"{i} towers: {pctg_visible:.1f} %")

                        doc.append("Average geodesic distance to closest visible tower is ")
                        doc.append(bold(f"{avg_visible:.1f} "))
                        doc.append(" kms")
                    with doc.create(MiniPage(width=r"0.68\textwidth")):
                        with doc.create(Figure(position='h!')) as fig:
                            fig.add_image("../aux_files/dist_distributions_visible.png", width=NoEscape(r"0.9\textwidth"))

    
                    doc.append(NoEscape(r"""
                    \begin{textblock*}{3cm}(10.1cm,8.8cm)  % {block width} (coords)
                    \includegraphics[width=.9\textwidth]{../aux_files/giga_logo.png}
                    \end{textblock*}
                    """))
                    doc.append(NewPage())
            else:
                print('Visibility cache not found! Report will not show visibility or allow P2P connectivity')
            
            #Fiber networks
            progress.value = 62
            progress_label.value = "62%"
            valid_schools = []
            if schools_to_connect == 'All schools':
                valid_schools = list(df_schools['giga_id_school'])
            elif schools_to_connect == 'Unconnected and unknown schools':
                for index,row in df_schools.iterrows():
                    if row['connectivity']!='No' and row['connectivity']!='no':
                        valid_schools.append(row['giga_id_school'])
            else:
                for index,row in df_schools.iterrows():
                    if row['connectivity']=='Yes' or row['connectivity']=='yes':
                        valid_schools.append(row['giga_id_school'])
            
            extra_fiber_nodes = []
            if fiber_node_types == 'Fiber nodes and connected schools':
                for index,row in df_schools.iterrows():
                    if row['connectivity']=='Yes' or row['connectivity']=='yes':
                        extra_fiber_nodes.append(row['giga_id_school'])
            elif fiber_node_types == 'Fiber nodes and connected schools with fiber':
                for index,row in df_schools.iterrows():
                    if row['type_connectivity']=='Fiber' or row['type_connectivity']=='fiber':
                        extra_fiber_nodes.append(row['giga_id_school'])
            
            Gs = []
            MSTs = []
            for i in range(len(fiber_thresholds)):
                Gs.append(get_fiber_graph(df_fiber,school_dists,fnode_dists,fiber_thresholds[i],valid_schools,extra_fiber_nodes))
                MSTs.append(get_MST(Gs[i]))
                
            fiber_lengths = []
            for i in range(len(fiber_thresholds)):
                fiber_lengths.append(get_total_fiber(MSTs[i],correction_coef))
            
            for i in range(len(fiber_thresholds)):
                plot_map_tree(gdf,df_schools,df_fiber,MSTs[i],fiber_thresholds[i],fiber_lengths[i],correction_coef)
    
            width = 1.0/len(fiber_thresholds)-0.05  
            with doc.create(Section('Fiber Network')):
                doc.append(VerticalSpace("5pt")) 
                doc.append(NoEscape(r'{\Large Fiber paths:}'))
                doc.append(VerticalSpace("2pt")) 
                with doc.create(Figure(position='h!')) as fig:
                    for i in range(len(fiber_thresholds)):
                        with doc.create(SubFigure(position='b', width=NoEscape(f'{width}'+r'\textwidth'))) as subfig:
                            subfig.add_image(f'../aux_files/fiber_paths_{fiber_thresholds[i]}.png', width=NoEscape(r'0.9\textwidth'))
                
    
                doc.append(NoEscape(r"""
                \begin{textblock*}{3cm}(10.1cm,8.8cm)  % {block width} (coords)
                \includegraphics[width=.9\textwidth]{../aux_files/giga_logo.png}
                \end{textblock*}
                """))
                doc.append(NewPage())
            
            #Connectivity scenarios
            with doc.create(Section('Connectivity scenarios')):
                doc.append(VerticalSpace("10pt")) 
                with doc.create(Enumerate()) as enum:
                    str1 = "Using "
                    str1 += r'\textbf{fiber}'
                    str1 += " connect to fiber nodes "
                    str1 += r'\textbf{all schools}'
                    str1 += " in the area around existing nodes given that the maximum fiber route distance between each of 2 schools is less than X km."
                    enum.add_item(NoEscape(str1))
                    str2 = "Using " 
                    str2 += r'\textbf{cellular modems}'
                    str2 += " connect to existing "
                    str2 += r'\textbf{4G mobile network}'
                    str2 += " all unconnected schools that are within strong 4G network coverage and not connected using fiber."
                    enum.add_item(NoEscape(str2))
                    str3 = "Using additional "
                    str3 += r'\textbf{point2point-microwave}'
                    str3 += " connect to 4G towers all unconnected schools that are "
                    str3 +=  r'\textbf{visible}'
                    str3 += " from 4G towers and not connected using fiber or existing 4G network."
                    enum.add_item(NoEscape(str3))
                    str4 = "Using "
                    str4 += r'\textbf{VSAT}'
                    str4 += " connect to "
                    str4 += r'\textbf{satellite}'
                    str4 += " all unconnected schools that are not connected using above mentioned options."
                    enum.add_item(NoEscape(str4))
    
                doc.append(NoEscape(r"""
                \begin{textblock*}{3cm}(10.1cm,8.8cm)  % {block width} (coords)
                \includegraphics[width=.9\textwidth]{../aux_files/giga_logo.png}
                \end{textblock*}
                """))
                doc.append(NewPage())
            
            #Connectivity scenarios
            progress.value = 66
            progress_label.value = "66%"
            cell_used = []
            p2p_used = []
            vsat_used = []
            for i in range(len(fiber_thresholds)):
                cell_used_i, p2p_used_i, vsat_used_i = fill_techs(df_schools,df_cell,cell_dists,cell_techs,visible_towers,MSTs[i],cell_strong,valid_schools)
                cell_used.append(cell_used_i)
                p2p_used.append(p2p_used_i)
                if sat_avail:
                    vsat_used.append(vsat_used_i)
                else:
                    vsat_used.append([])
                plot_map_4(gdf,df_schools,MSTs[i],cell_used[i],p2p_used[i],vsat_used[i],fiber_thresholds[i])
                
            roman = ['II','III','IV','V']
            for i in range(len(fiber_thresholds)):
                th = fiber_thresholds[i]
                mds = th*correction_coef
                with doc.create(Section('Connectivity Scenarios ' + roman[i])):
                    doc.append(VerticalSpace("5pt")) 
                    doc.append(NoEscape(r'{\Large School Technologies:}'))
                    doc.append(NoEscape(r'\\'))  # Start a new line
                    doc.append(VerticalSpace("2pt")) 
                    with doc.create(MiniPage(width=r"0.3\textwidth")):
                        doc.append("The maximum fiber route distance between 2 schools is ")
                        doc.append(bold(f"{mds:.1f} kms"))
                    with doc.create(MiniPage(width=r"0.68\textwidth")):
                        with doc.create(Figure(position='h!')) as fig:
                            fig.add_image('../aux_files/scenarios_'+str(th)+'.png', width=NoEscape(r'0.8\textwidth'))
                
                    doc.append(NoEscape(r"""
                    \begin{textblock*}{3cm}(10.1cm,8.8cm)  % {block width} (coords)
                    \includegraphics[width=.9\textwidth]{../aux_files/giga_logo.png}
                    \end{textblock*}
                    """))
                    doc.append(NewPage())
                    
            #Connectivity scenarios
            progress.value = 70
            progress_label.value = "70%"
            cell_used_0, p2p_used_0, vsat_used_0 = fill_techs_no_fiber(df_schools,df_cell,cell_dists,cell_techs,visible_towers,cell_strong,valid_schools)
            if not sat_avail:
                vsat_used_0 = []
            xs_techs = {'Fiber':[0],'cellular':[0],'P2P':[0],'satellite':[0]}
            ys_techs = {'Fiber':[0],'cellular':[len(cell_used_0)],'P2P':[len(p2p_used_0)],'satellite':[len(vsat_used_0)]}

            for i in range(len(fiber_thresholds)):
                xs_techs['Fiber'].append(fiber_thresholds[i]*correction_coef)
                xs_techs['cellular'].append(fiber_thresholds[i]*correction_coef)
                xs_techs['P2P'].append(fiber_thresholds[i]*correction_coef)
                xs_techs['satellite'].append(fiber_thresholds[i]*correction_coef)
                ys_techs['Fiber'].append(len(valid_schools)-(len(cell_used[i])+len(p2p_used[i])+len(vsat_used[i])))
                ys_techs['cellular'].append(len(cell_used[i]))
                ys_techs['P2P'].append(len(p2p_used[i]))
                ys_techs['satellite'].append(len(vsat_used[i]))
            
            plot_connectivity_scenarios(xs_techs,ys_techs,len(valid_schools)+200)
            with doc.create(Section('Technologies Distribution')):
                doc.append(VerticalSpace("5pt")) 
                doc.append(NoEscape(r'{\Large Distribution of schools by technologies:}'))
                doc.append(NoEscape(r'\\'))  # Start a new line
                doc.append(VerticalSpace("2pt")) 
                with doc.create(MiniPage(width=r"0.3\textwidth")):
                    doc.append("The influence of the maximum fiber link distance on technology distribution")
                with doc.create(MiniPage(width=r"0.68\textwidth")):
                    with doc.create(Figure(position='h!')) as fig:
                        fig.add_image('../aux_files/schools_techs.png', width=NoEscape(r'0.8\textwidth'))
                
                doc.append(NoEscape(r"""
                \begin{textblock*}{3cm}(10.1cm,8.8cm)  % {block width} (coords)
                \includegraphics[width=.9\textwidth]{../aux_files/giga_logo.png}
                \end{textblock*}
                """))
                doc.append(NewPage())
                
            #Cost scenarios
            progress.value = 74
            progress_label.value = "74%"
            xs_costs = {'Fiber':[0],'cellular':[0],'P2P':[0],'satellite':[0]}
            xs_tech_costs = {'Fiber':[0],'cellular':[0],'P2P':[0],'satellite':[0]}
            xs_elec_costs = {'Fiber':[0],'cellular':[0],'P2P':[0],'satellite':[0]}
            cell_costs_0,cell_elec_costs_0 = get_cell_costs(df_schools,cell_used_0,years_opex,min_bandwidth)
            p2p_costs_0,p2p_elec_costs_0 = get_p2p_costs(df_schools,p2p_used_0,years_opex,min_bandwidth)
            vsat_costs_0,vsat_elec_costs_0 = get_vsat_costs(df_schools,vsat_used_0,years_opex,min_bandwidth)
            ys_costs = {'Fiber':[0],'cellular':[cell_costs_0+cell_elec_costs_0],'P2P':[p2p_costs_0+p2p_elec_costs_0],'satellite':[vsat_costs_0+vsat_elec_costs_0]}
            ys_tech_costs = {'Fiber':[0],'cellular':[cell_costs_0],'P2P':[p2p_costs_0],'satellite':[vsat_costs_0]}
            ys_elec_costs = {'Fiber':[0],'cellular':[cell_elec_costs_0],'P2P':[p2p_elec_costs_0],'satellite':[vsat_elec_costs_0]}

            for i in range(len(fiber_thresholds)):
                xs_costs['Fiber'].append(fiber_thresholds[i]*correction_coef)
                xs_costs['cellular'].append(fiber_thresholds[i]*correction_coef)
                xs_costs['P2P'].append(fiber_thresholds[i]*correction_coef)
                xs_costs['satellite'].append(fiber_thresholds[i]*correction_coef)
    
                xs_tech_costs['Fiber'].append(fiber_thresholds[i]*correction_coef)
                xs_tech_costs['cellular'].append(fiber_thresholds[i]*correction_coef)
                xs_tech_costs['P2P'].append(fiber_thresholds[i]*correction_coef)
                xs_tech_costs['satellite'].append(fiber_thresholds[i]*correction_coef)
    
                xs_elec_costs['Fiber'].append(fiber_thresholds[i]*correction_coef)
                xs_elec_costs['cellular'].append(fiber_thresholds[i]*correction_coef)
                xs_elec_costs['P2P'].append(fiber_thresholds[i]*correction_coef)
                xs_elec_costs['satellite'].append(fiber_thresholds[i]*correction_coef)
    
                fiber_costs_i,fiber_elec_costs_i = get_fiber_costs(df_schools,MSTs[i],years_opex,correction_coef,min_bandwidth)
                cell_costs_i,cell_elec_costs_i = get_cell_costs(df_schools,cell_used[i],years_opex,min_bandwidth)
                p2p_costs_i,p2p_elec_costs_i = get_p2p_costs(df_schools,p2p_used[i],years_opex,min_bandwidth)
                vsat_costs_i,vsat_elec_costs_i = get_vsat_costs(df_schools,vsat_used[i],years_opex,min_bandwidth)
    
                ys_costs['Fiber'].append(fiber_costs_i+fiber_elec_costs_i)
                ys_costs['cellular'].append(cell_costs_i+cell_elec_costs_i)
                ys_costs['P2P'].append(p2p_costs_i+p2p_elec_costs_i)
                ys_costs['satellite'].append(vsat_costs_i+vsat_elec_costs_i)
    
                ys_tech_costs['Fiber'].append(fiber_costs_i)
                ys_tech_costs['cellular'].append(cell_costs_i)
                ys_tech_costs['P2P'].append(p2p_costs_i)
                ys_tech_costs['satellite'].append(vsat_costs_i)
    
                ys_elec_costs['Fiber'].append(fiber_elec_costs_i)
                ys_elec_costs['cellular'].append(cell_elec_costs_i)
                ys_elec_costs['P2P'].append(p2p_elec_costs_i)
                ys_elec_costs['satellite'].append(vsat_elec_costs_i)
            maxy = max(max(ys_costs['Fiber']),max(ys_costs['P2P']),max(ys_costs['cellular']),max(ys_costs['satellite']))
            plot_costs(xs_costs,ys_costs,maxy,'total')
            with doc.create(Section('Technology Cost Distribution')):
                doc.append(VerticalSpace("5pt")) 
                doc.append(NoEscape(r'{\Large Distribution of costs by technologies:}'))
                doc.append(NoEscape(r'\\'))  # Start a new line
                doc.append(VerticalSpace("2pt")) 
                with doc.create(MiniPage(width=r"0.3\textwidth")):
                    doc.append("The influence of the maximum fiber link distance on technology cost distribution")
                with doc.create(MiniPage(width=r"0.68\textwidth")):
                    with doc.create(Figure(position='h!')) as fig:
                        fig.add_image('../aux_files/schools_costs_total.png', width=NoEscape(r'0.8\textwidth'))
                
                doc.append(NoEscape(r"""
                \begin{textblock*}{3cm}(10.1cm,8.8cm)  % {block width} (coords)
                \includegraphics[width=.9\textwidth]{../aux_files/giga_logo.png}
                \end{textblock*}
                """))
                doc.append(NewPage())
                
            #Cost scenarios 2
            progress.value = 78
            progress_label.value = "78%"
            maxy = max(max(ys_tech_costs['Fiber']),max(ys_tech_costs['P2P']),max(ys_tech_costs['cellular']),max(ys_tech_costs['satellite']))
            plot_costs(xs_tech_costs,ys_tech_costs,maxy,'technology')
            maxy = max(max(ys_elec_costs['Fiber']),max(ys_elec_costs['P2P']),max(ys_elec_costs['cellular']),max(ys_elec_costs['satellite']))
            plot_costs(xs_elec_costs,ys_elec_costs,maxy,'electricity')
            with doc.create(Section('Technology Cost Distribution II')):
                doc.append(VerticalSpace("5pt")) 
                doc.append(NoEscape(r'{\Large Distribution of costs - technology and electricity:}'))
                doc.append(NoEscape(r'\\'))  # Start a new line
                doc.append(VerticalSpace("10pt")) 
                with doc.create(MiniPage(width=r"0.45\textwidth")):
                    with doc.create(Figure(position='h!')) as fig:
                        fig.add_image('../aux_files/schools_costs_technology.png', width=NoEscape(r'0.9\textwidth'))
                with doc.create(MiniPage(width=r"0.45\textwidth")):
                    with doc.create(Figure(position='h!')) as fig:
                        fig.add_image('../aux_files/schools_costs_electricity.png', width=NoEscape(r'0.9\textwidth'))
                
                doc.append(NoEscape(r"""
                \begin{textblock*}{3cm}(10.1cm,8.8cm)  % {block width} (coords)
                \includegraphics[width=.9\textwidth]{../aux_files/giga_logo.png}
                \end{textblock*}
                """))
                doc.append(NewPage())
                
            #Cost final
            progress.value = 82
            progress_label.value = "82%"
            fiber_th_final = tabs.children[0].children[1].children[1].value/correction_coef
            cell_th_final = tabs.children[1].children[1].children[1].value
            Gf = get_fiber_graph(df_fiber,school_dists,fnode_dists,fiber_th_final,valid_schools,extra_fiber_nodes)
            MSTf = get_MST(Gf)
            if not sat_avail:
                vsat_used_f = []
            cell_used_f, p2p_used_f, vsat_used_f = fill_techs(df_schools,df_cell,cell_dists,cell_techs,visible_towers,MSTf,cell_th_final,valid_schools)
            df_school_costs = calculate_all_school_costs(df_schools,MSTf,cell_used_f, p2p_used_f, vsat_used_f,valid_schools,correction_coef,years_opex,min_bandwidth)
            plot_school_costs_gradient(gdf,df_school_costs)
            total_cost = df_school_costs['total_cost'].sum()
            with doc.create(Section('School costs')):
                doc.append(VerticalSpace("5pt")) 
                doc.append(NoEscape(r'{\Large Distribution of school costs:}'))
                doc.append(NoEscape(r'\\'))  # Start a new line
                doc.append(VerticalSpace("2pt")) 
                with doc.create(MiniPage(width=r"0.3\textwidth")):
                    doc.append(f"The total cost of connecting the schools is {total_cost:.2f} USD")
                with doc.create(MiniPage(width=r"0.68\textwidth")):
                    with doc.create(Figure(position='h!')) as fig:
                        fig.add_image('../aux_files/school_costs_gradient.png', width=NoEscape(r'0.7\textwidth'))
                
                doc.append(NoEscape(r"""
                \begin{textblock*}{3cm}(10.1cm,8.8cm)  % {block width} (coords)
                \includegraphics[width=.9\textwidth]{../aux_files/giga_logo.png}
                \end{textblock*}
                """))
                doc.append(NewPage())
                
            #Cost final 2
            progress.value = 86
            progress_label.value = "86%"
            tech_capex = df_school_costs['capex'].sum()
            tech_opex = df_school_costs['opex'].sum()
            elec_capex = df_school_costs['electricity_capex'].sum()
            elec_opex = df_school_costs['electricity_opex'].sum()
            total_capex = df_school_costs['total_capex'].sum()
            total_opex = df_school_costs['total_opex'].sum()

            capex = (tech_capex,elec_capex,total_capex)
            opex = (tech_opex,elec_opex,total_opex)
            plot_capex_opex(capex,opex)
            with doc.create(Section('School costs II')):
                doc.append(VerticalSpace("5pt")) 
                doc.append(NoEscape(r'{\Large Capex/opex of school costs:}'))
                doc.append(NoEscape(r'\\'))  # Start a new line
                doc.append(VerticalSpace("2pt")) 
                with doc.create(Figure(position='h!')) as fig:
                    fig.add_image('../aux_files/capex_opex.png', width=NoEscape(r'0.6\textwidth'))
                
                doc.append(NoEscape(r"""
                \begin{textblock*}{3cm}(10.1cm,8.8cm)  % {block width} (coords)
                \includegraphics[width=.9\textwidth]{../aux_files/giga_logo.png}
                \end{textblock*}
                """))
                doc.append(NewPage())
            
            
            #compile report
            #print('compile report')
            progress.value = 90
            progress_label.value = "90%"
            fn_name = './'+iso3code+"/"+iso3code+"_report"
            doc.generate_pdf(fn_name, clean_tex=False, clean = True)
            #########################
            
            ### remove aux files and copy report to blob
            if os.path.exists(plots_directory):
                shutil.rmtree(plots_directory)
                
            blob_name = 'source/infra/reports/'+iso3code+'_report.pdf'
            blob_client = container_client.get_blob_client(blob_name)

            # Upload the file
            with open(fn_name+".pdf", "rb") as data:
                blob_client.upload_blob(data,overwrite=True)
    
    
            progress.value = 100
            progress_label.value = "100%"
            
            
            #display link
            file_path = fn_name+".pdf"
            filename = os.path.basename(file_path)
            data = open(file_path, "rb").read()
            b64 = base64.b64encode(data)
            payload = b64.decode()
            
            
            
            # Create HTML link
            title = "Download Report"
            html = '<a download="{filename}" href="data:text/csv;base64,{payload}" target="_blank">{title}</a>'
            html = html.format(payload=payload,title=title,filename=filename)
            display(HTML(html))
            
            csv = df_school_costs.to_csv()
            b64 = base64.b64encode(csv.encode())
            payload = b64.decode()
            
            filename=iso3code+"_school_costs.csv"
            title = "Download school costs table" 
            html = '<a download="{filename}" href="data:text/csv;base64,{payload}" target="_blank">{title}</a>'
            html = html.format(payload=payload, title=title, filename=filename)
            display(HTML(html))
            
        else:
            sys.stdout.write('You need to select a country file!')
            sys.stdout.flush()

buttonR.on_click(on_button_clickedR)

Button(description='Generate Report', style=ButtonStyle())

Output()

## Click the button below to start over

In [ ]:
buttonReset = widgets.Button(description="Reset!")
outputReset = widgets.Output()
display(buttonReset,outputReset)

def on_button_clickedReset(b):
    
    with outputReset:
        global iso3code
        global cell_techs
        global visible_towers

        clear_output(wait=False)

        #enable widgets
        buttonR.disabled = False
        #wBlob.disabled = False
        wCF.disabled = False
        wFCC.disabled = False
        for i in range(0, len(checkboxesF)):
            checkboxesF[i].disabled = False
        for i in range(0, len(checkboxesFP)):
            checkboxesFP[i].disabled = False
        wCS.disabled = False
        for i in range(0, len(checkboxesC)):
            checkboxesC[i].disabled = False
        for i in range(0, len(checkboxesVB)):
            checkboxesVB[i].disabled = False
                
        wP2TH.disabled = False
        wP2AH.disabled = False
            
        for i in range(0, len(checkboxesCT)):
            checkboxesCT[i].disabled = False
                
        wP2TW.disabled = False
            
        wYO.disabled = False
        wMB.disabled = False
            
        wSCT.disabled = False
        wFNT.disabled = False
            
        for i in range(0, len(checkboxesP2P)):
            checkboxesP2P[i].disabled = False
        for i in range(0, len(checkboxesP2T)):
            checkboxesP2T[i].disabled = False
                
        wSATa.disabled = False
        change_disabled_tabs(False)
        
        #reset vars 
        iso3code = None
        cell_techs = ['LTE'] #for 4G
        visible_towers = {}

        sys.stdout.write('Done!')
        sys.stdout.flush()
        
    for output in all_outputs:
        with output:
            clear_output(wait=False)
            
    wCF.value = "None"
        
buttonReset.on_click(on_button_clickedReset)